In [ ]:
%%writefile src/data/load_data.py
import pandas as pd
import numpy as np

def load_data(path='data/Research Data Project/Research Data Project/exit_velo_project_data.csv'):
    """
    Load raw exit velocity data from the data directory.
    
    Returns:
        pd.DataFrame: Raw exit velocity data
    """
    # This is a placeholder - in a real implementation, you would load actual data
    # For example: df = pd.read_csv('data/raw/exit_velo.csv')

    df = pd.read_csv(path)
    return df 



# ──  utility ─────────────────────────────────────────────────────────
def clean_raw(df: pd.DataFrame, 
              target: str = "exit_velo"
              ,debug: bool = False) -> pd.DataFrame:
    """
    Central place to:
      1. Drop rows with NaN in *target*.
      2. Print a concise null‑summary afterwards (one line per column).
    
    Returns a *fresh copy* (never mutates in‑place).
    """
    if debug:
        print(f"Dropping rows with NaN in {target}")
        print(df.isna().sum())
        
    drop_columns = [target, "hit_type", "launch_angle"]
    # remove 125 hit_type rows (~0.01%) and 2 launch_angle rows (~0.00%) after the target was dropped because that's automatically done
    out = df.dropna(subset=drop_columns).copy()   

    if debug:
        print(f"after rows dropped with NaN in {target}")
        print(out.isna().sum())
    # quick dashboard
    nulls = out.isna().sum()
    non_zero = nulls[nulls > 0]
    if non_zero.empty:
        print(f"✅  After target‑filter, no other nulls (n={len(out):,}).")
    else:
        print("⚠️  Nulls after target‑filter:")
        for col, cnt in non_zero.items():
            pct = cnt / len(out)
            print(f"  • {col:<15} {cnt:>7,} ({pct:5.2%})")

    return out




def load_and_clean_data(path='data/Research Data Project/Research Data Project/exit_velo_project_data.csv'
                        ,debug: bool = False):
    df = load_data(path)
    df = clean_raw(df, debug = True) #hangtime only one left to impute
    return df


if __name__ == "__main__":
    path = 'data/Research Data Project/Research Data Project/exit_velo_project_data.csv'
    # df = load_data(path)
    # print(df.head())
    # print(df.columns)
    # df = clean_raw(df, debug = True) #hangtime only one left to impute

    df = load_and_clean_data(path, debug = True)
    print(df.head())
    print(df.columns)
    

# Feature Engineering

In [ ]:
%%writefile src/features/feature_engineering.py
"""Feature engineering utilities for the exit‑velo project.

All helpers are pure functions that take a pandas DataFrame and return a *copy*
with additional engineered columns so we avoid side effects.

Example
-------
>>> from src.data.load_data import load_and_clean_data
>>> from src.features.feature_engineering import feature_engineer
>>> df = load_and_clean_data()
>>> df_fe = feature_engineer(df)
"""
from __future__ import annotations

import pandas as pd
import numpy as np

###############################################################################
# Helper functions
###############################################################################


def _rolling_stat_lagged(
    df: pd.DataFrame,
    group_cols: list[str],
    target: str,
    stat: str = "mean",
    window: int = 50,
) -> pd.Series:
    """
    Group-wise rolling statistic using only *previous* rows.
    For each group defined by group_cols, shift the target by 1 row 
    then compute a rolling(window) agg(stat) with min_periods=10.
    """
    # 1) Within each group, shift the target by one so we only use past data
    def shifted_rolling(x: pd.Series) -> pd.Series:
        return x.shift(1).rolling(window=window, min_periods=10).agg(stat)

    rolled = (
        df
        .groupby(group_cols)[target]     # group by batter_id or pitcher_id
        .apply(shifted_rolling)          # shift & roll inside each group
        .reset_index(level=group_cols, drop=True)  # get back a plain Series
    )
    return rolled


###############################################################################
# Public API
###############################################################################

def feature_engineer(df: pd.DataFrame, copy: bool = True) -> pd.DataFrame:
    """Return a DataFrame enriched with engineered features (no leakage)."""

    if copy:
        df = df.copy()

    # 1) Uppercase strings
    str_cols = df.select_dtypes(include=['object', 'string']).columns
    df[str_cols] = df[str_cols].apply(lambda col: col.str.upper())

    # 2) Age
    df["age_sq"]   = df["age"] ** 2
    df["age_bin"]  = pd.qcut(df["age"], q=4, duplicates='drop')

    # 3) Height
    avg_height    = df["batter_height"].mean()
    df["height_diff"] = df["batter_height"] - avg_height

    # 4) Launch & spray bins
    df["la_bin"]    = pd.qcut(df["launch_angle"], q=5, duplicates='drop')
    df["spray_bin"] = pd.qcut(df["spray_angle"], q=3, duplicates='drop')

    # 5) Handedness & matchups
    df["same_hand"]        = (df["batter_hand"] == df["pitcher_hand"])
    df["hand_match"]       = df["batter_hand"] + "_VS_" + df["pitcher_hand"]
    df["pitch_hand_match"] = df["pitch_group"] + "_" + df["hand_match"]

    # 6) Batter lagged rolling EV stats
    df["player_ev_mean50"] = _rolling_stat_lagged(df, ["batter_id"], "exit_velo", "mean", 50)
    df["player_ev_std50"]  = _rolling_stat_lagged(df, ["batter_id"], "exit_velo",  "std", 50)
    gm = df["exit_velo"].mean(); gs = df["exit_velo"].std()
    df["player_ev_mean50"].fillna(gm, inplace=True)
    df["player_ev_std50"].fillna(gs, inplace=True)

    # 7) Pitcher lagged mean
    df["pitcher_ev_mean50"] = _rolling_stat_lagged(df, ["pitcher_id"], "exit_velo", "mean", 50)
    df["pitcher_ev_mean50"].fillna(gm, inplace=True)

    # 8) Center covariates
    df["age_centered"]    = df["age"]    - df["age"].median()
    df["season_centered"] = df["season"] - df["season"].median()
    df["level_idx"]       = df["level_abbr"].map({"AA":0, "AAA":1, "MLB":2})

    return df

###############################################################################
# CLI entry‑point (quick smoke test)
###############################################################################

if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path, debug = True)
    print(df.head())
    print(df.columns)

    df_fe = feature_engineer(df)

    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    print(df_fe.columns)
    
    # --- inspect nulls in the raw data ---
    null_counts = df_fe.isnull().sum()
    print(null_counts)
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")

# ColumnSchema: separates raw and engineered columns

In [ ]:
%%writefile src/data/ColumnSchema.py
""" 
Column schema helper for exit‑velo project.

Centralises every raw and engineered column name in one place and exposes
 type‑safe accessors so downstream code never hard‑codes strings.

Usage
-----
>>> from src.features.columns import cols
>>> num_cols = cols.numerical()
>>> ord_cols = cols.ordinal()
>>> all_for_model = cols.model_features()
"""

from functools import lru_cache
from typing import List, Dict
import json

class _ColumnSchema:
    """Container for canonical column lists.

    Keeping everything behind methods avoids accidental mutation and lets
    IDEs offer autocompletion (because the return type is always `List[str]`).
    """

    _ID_COLS: List[str] = [
        "season", "batter_id", "pitcher_id",
    ]

    _ORDINAL_CAT_COLS: List[str] = [
        "level_abbr",   # AA < AAA < MLB
        "age_bin",      # 4 quantile bins of age
        "la_bin",       # 5 quantile bins of launch angle
        "spray_bin",    # 3 quantile bins of spray angle
        # "outcome",      # categorical outcome (out, single, double, triple, home run, sacrifice fly, sacrifice bunt, fielders choice)
    ]


    _NOMINAL_CAT_COLS = [
        "hit_type",
        "pitch_group",
        "batter_hand",
        "pitcher_hand",
        "hand_match",
        "pitch_hand_match",
        "same_hand",
    ]

    _NUMERICAL_COLS = [
        "launch_angle",
        "spray_angle",
        "hangtime",
        "height_diff",
        "age_sq",
        "age_centered",
        "season_centered",
        "level_idx",
        "player_ev_mean50",
        "player_ev_std50",
        "pitcher_ev_mean50",
    ]


    _TARGET_COL: str = "exit_velo"

    # ────────────────────────────────────────────────────────────────────
    # Public helpers
    # ────────────────────────────────────────────────────────────────────
    def id(self) -> List[str]:
        return self._ID_COLS.copy()

    def ordinal(self) -> List[str]:
        return self._ORDINAL_CAT_COLS.copy()

    def nominal(self) -> List[str]:
        return self._NOMINAL_CAT_COLS.copy()

    def categorical(self) -> List[str]:
        """All cat cols (ordinal + nominal)."""
        return self._ORDINAL_CAT_COLS + self._NOMINAL_CAT_COLS

    def numerical(self) -> List[str]:
        return self._NUMERICAL_COLS.copy()


    def target(self) -> str:
        return self._TARGET_COL

    # ------------------------------------------------------------------
    @lru_cache(maxsize=1)
    def model_features(self) -> List[str]:
        """Columns fed into the ML pipeline *after* preprocess.

        Excludes the target but includes derived cols.
        """
        phys_minus_target = [
            c for c in self._NUMERICAL_COLS if c != self._TARGET_COL
        ]

        return (
            phys_minus_target
            + self._NOMINAL_CAT_COLS
            + self._ORDINAL_CAT_COLS  # some algos want raw string order
        )

    def all_raw(self) -> List[str]:
        """Returns every column expected in raw input CSV (incl. engineered)."""
        return (
            self._ID_COLS
            + self._ORDINAL_CAT_COLS
            + self._NOMINAL_CAT_COLS
            + self._NUMERICAL_COLS
        )

    def as_dict(self) -> Dict[str, List[str]]:
        """Dictionary form – handy for YAML/JSON dumps."""
        return {
            "id": self.id(),
            "ordinal": self.ordinal(),
            "nominal": self.nominal(),
            "numerical": self.numerical(),
            "target": [self.target()],
        }


if __name__ == "__main__":
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Target column:      ", cols.target())
    print("Model features:     ", cols.model_features())
    print("All raw columns:    ", cols.all_raw())
    print("\nAs dict (JSON):")
    print(json.dumps(cols.as_dict(), indent=2))



In [ ]:
%%writefile src/features/eda.py

import pandas as pd  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import numpy as np  # type: ignore
from src.data.ColumnSchema import _ColumnSchema
from pandas.api.types import is_categorical_dtype

# Optional imports with fallbacks for advanced statistics
try:
    import scipy.stats as stats  # type: ignore
    from statsmodels.nonparametric.smoothers_lowess import (
        lowess  # type: ignore
    )
    from statsmodels.stats.stattools import (
        durbin_watson  # type: ignore
    )
    _HAS_STATS_LIBS = True
except ImportError:
    _HAS_STATS_LIBS = False
    print("Warning: scipy or statsmodels not available. "
          "Some diagnostics will be limited.")

from scipy.stats import f_oneway, ttest_ind



def get_column_groups() -> dict:
    """
    Return a mapping of column-type → list of columns,
    based on the canonical schema in src.features.feature_selection.cols.
    """
    return cols.as_dict()

def check_nulls(df: pd.DataFrame):
    # Identify columns with null values
    null_columns = df.columns[df.isnull().any()].tolist()
    
    # Output the columns with null values
    if null_columns:
        print("Columns with null values:", null_columns)
    else:
        print("No columns with null values.")


def quick_pulse_check(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    group_col: str = "batter_id",
    level_col: str = "level_abbr"
) -> pd.DataFrame:
    """
    Print a quick summary table:
      - total rows
      - unique batters
      - overall median exit_velo
      - median exit_velo by level
      - distribution of events per batter (median, 25th pct)
      - distribution of seasons per batter
      - pearson correlations of velo with launch_angle & hangtime
    Returns a pd.DataFrame with those metrics.
    """
    df = df.copy()
    total_rows = len(df)
    n_batters = df[group_col].nunique()
    overall_med = df[velo_col].median()

    # median by level
    med_by_level = df.groupby(level_col)[velo_col].median()

    # events per batter
    ev_per = df[group_col].value_counts()
    ev_stats = ev_per.quantile([0.25, 0.5]).to_dict()

    # seasons per batter
    seasons_per = df.groupby(group_col)["season"].nunique()
    seasons_stats = seasons_per.value_counts().sort_index().to_dict()

    # basic correlations
    corr = df[[velo_col, "launch_angle", "hangtime"]].corr()[velo_col].drop(velo_col)

    # Build a summary table
    metrics = [
        "Total rows",
        "Unique batters",
        "Overall median EV",
    ]
    values = [
        total_rows,
        n_batters,
        overall_med,
    ]
    
    # Add level-specific metrics
    for lvl in med_by_level.index:
        metrics.append(f"Median EV @ {lvl}")
        values.append(med_by_level[lvl])
    
    # Add batter event metrics
    metrics.extend([
        "Events per batter (25th pct)",
        "Events per batter (median)",
    ])
    values.extend([
        ev_stats.get(0.25, "N/A"),
        ev_stats.get(0.5, "N/A"),
    ])
    
    # Add season distribution
    for season_count, count in seasons_stats.items():
        metrics.append(f"Batters with {season_count} season(s)")
        values.append(count)
    
    # Add correlations
    metrics.extend([
        "ρ(exit_velo, launch_angle)",
        "ρ(exit_velo, hangtime)",
    ])
    values.extend([
        corr.get("launch_angle", "N/A"),
        corr.get("hangtime", "N/A"),
    ])

    table = pd.DataFrame({
        "Metric": metrics,
        "Value": values
    })
    
    print(table.to_string(index=False))
    return table


def red_flag_small_samples(df: pd.DataFrame,
                           group_col: str = "batter_id",
                           threshold: int = 15) -> pd.Series:
    """
    Identify batters with fewer than `threshold` events.
    Returns a Series of counts indexed by batter_id.
    """
    counts = df[group_col].value_counts()
    small = counts[counts < threshold]
    print(f"> Batters with fewer than {threshold} events: {len(small)}")
    if len(small) > 0:
        print(f"  First few: {', '.join(map(str, small.index[:5]))}")
    return small


def red_flag_level_effect(df: pd.DataFrame,
                          level_col: str = "level_abbr",
                          velo_col: str = "exit_velo") -> tuple:
    """
    One-way ANOVA of exit_velo across levels.
    Returns (F-statistic, p-value) or (None, None) if scipy is not available.
    """
    if not _HAS_STATS_LIBS:
        print("> ANOVA on exit_velo by level: scipy not available")
        print("> Basic level summary instead:")
        summary = df.groupby(level_col)[velo_col].agg(['mean', 'std', 'count'])
        print(summary)
        return None, None
    
    groups = [
        df[df[level_col] == lvl][velo_col].dropna()
        for lvl in df[level_col].unique()
    ]
    F, p = stats.f_oneway(*groups)
    print(f"> ANOVA on {velo_col} by {level_col}: F={F:.3f}, p={p:.3e}")
    return F, p


# ------------------------------------------------------------------
#  REPLACE old red_flag_level_effect  → clearer name & doc
# ------------------------------------------------------------------
def league_level_effect(
    df: pd.DataFrame,
    level_col: str = "level_abbr",
    velo_col: str = "exit_velo",
) -> tuple[float | None, float | None]:
    """
    🔹 Why it matters – confirms MLB vs Triple‑A (etc.) differences to
      justify hierarchical level effects in the model.

    One‑way ANOVA of `exit_velo` across `level_col`.
    Returns (F, p) or (None, None) if SciPy unavailable.
    """
    if not _HAS_STATS_LIBS:
        print("> SciPy unavailable – falling back to group summary")
        print(df.groupby(level_col)[velo_col].describe())
        return None, None

    groups = [df[df[level_col] == lv][velo_col].dropna()
              for lv in df[level_col].unique()]
    f_val, p_val = stats.f_oneway(*groups)
    print(f"> Level effect ANOVA: F={f_val:.3f}, p={p_val:.3e}")
    return f_val, p_val



def diag_age_effect(df: pd.DataFrame,
                    age_col: str = "age_centered",
                    velo_col: str = "exit_velo") -> np.ndarray | None:
    """
    LOWESS smoothing of exit_velo vs. age_centered.
    Returns the smoothed array or None if statsmodels is not available.
    """
    if not _HAS_STATS_LIBS:
        print("> Age effect analysis: statsmodels not available")
        print("> Basic correlation instead:")
        corr = df[[age_col, velo_col]].corr().iloc[0, 1]
        print(f"Correlation between {age_col} and {velo_col}: {corr:.3f}")
        return None
    
    # Run LOWESS smoothing
    smooth_result = lowess(df[velo_col], df[age_col])
    
    # Plot the result
    plt.figure(figsize=(6, 3))
    plt.scatter(df[age_col], df[velo_col], alpha=0.1, s=1, color='gray')
    plt.plot(
        smooth_result[:, 0], 
        smooth_result[:, 1], 
        'r-', 
        linewidth=2, 
        label="LOWESS fit"
    )
    plt.xlabel(age_col)
    plt.ylabel(velo_col)
    plt.title("Age effect (LOWESS)")
    plt.legend()
    plt.tight_layout()
    
    return smooth_result


def diag_time_series_dw(
    df: pd.DataFrame,
    time_col: str = "season",
    group_col: str = "batter_id",
    velo_col: str = "exit_velo"
) -> pd.Series | None:
    """
    Compute Durbin–Watson on each batter's time series of mean exit_velo.
    Returns a Series of DW statistics or None if statsmodels is not available.
    """
    if not _HAS_STATS_LIBS:
        print("> Time series analysis: statsmodels not available")
        return None
    
    # Create pivot table of seasons (columns) by batters (rows)
    pivot = (
        df
        .groupby([group_col, time_col])[velo_col]
        .mean()
        .unstack(fill_value=np.nan)
    )
    
    # Only process batters with at least 3 seasons
    valid_batters = pivot.dropna(thresh=3).index
    if len(valid_batters) == 0:
        print("> No batters with sufficient seasons for Durbin-Watson test")
        return None
    
    # Calculate DW statistic for each valid batter
    dw_stats = {}
    for batter in valid_batters:
        series = pivot.loc[batter].dropna()
        if len(series) >= 3:  # Recheck after dropna
            dw = durbin_watson(series)
            dw_stats[batter] = dw
    
    dw_series = pd.Series(dw_stats)
    print(
        f"> Mean Durbin–Watson across {len(dw_series)} batters: "
        f"{dw_series.mean():.3f}"
    )
    print("> DW < 1.5 suggests positive autocorrelation")
    print("> DW > 2.5 suggests negative autocorrelation")
    print("> DW ≈ 2.0 suggests no autocorrelation")
    
    return dw_series


# ------------------------------------------------------------------
#  REPLACE old diag_time_series_dw WITH optional helper
# ------------------------------------------------------------------
def _optional_dw_check(
    df: pd.DataFrame,
    time_col: str = "season",
    group_col: str = "batter_id",
    velo_col: str = "exit_velo",
) -> pd.Series | None:
    """
    (OPTIONAL) Durbin–Watson residual autocorrelation **per batter**.
    Mostly irrelevant for cross‑sectional EV analysis but retained
    behind a private name for power users.
    """
    if not _HAS_STATS_LIBS:
        return None
    pivot = (
        df.groupby([group_col, time_col])[velo_col]
          .mean().unstack()
    )
    stats_out = {}
    for idx, row in pivot.dropna(thresh=3).iterrows():
        if row.count() >= 3:
            stats_out[idx] = durbin_watson(row.dropna())
    if not stats_out:
        print("> DW check: no eligible batters")
        return None
    s = pd.Series(stats_out)
    print(f"DW mean={s.mean():.2f} (1.5<→pos autocorr, >2.5→neg)")
    return s




def check_red_flags(df: pd.DataFrame, 
                    sample_threshold: int = 15) -> dict:
    """
    Run all red flag checks and return the results in a dictionary.
    """
    results = {}
    
    # Check for small sample sizes
    small_samples = red_flag_small_samples(df, threshold=sample_threshold)
    results['small_samples'] = small_samples
    
    # Check for level effects
    f_stat, p_val = red_flag_level_effect(df)
    results['level_effect'] = {
        'f_statistic': f_stat,
        'p_value': p_val
    }
    
    return results


def plot_distributions(df: pd.DataFrame,
                       velo_col: str = "exit_velo",
                       by: str = "level_abbr"):
    """
    Histogram of `velo_col` faceted by `by`.
    Returns the Matplotlib figure so callers can save or show it.
    """
    groups = df[by].unique()
    fig, axes = plt.subplots(len(groups), 1,
                             figsize=(6, 2.8 * len(groups)),
                             sharex=True)
    for ax, grp in zip(axes, groups):
        ax.hist(df[df[by] == grp][velo_col], bins=30, alpha=0.75)
        ax.set_title(f"{by} = {grp} (n={len(df[df[by] == grp])})")
        ax.set_xlabel(velo_col)
    fig.tight_layout()
    return fig


def plot_correlations(df: pd.DataFrame, cols: list[str]):
    """
    Heat-map of Pearson correlations for `cols`.
    """
    corr = df[cols].corr()
    fig, ax = plt.subplots(figsize=(0.6 * len(cols) + 2,
                                    0.6 * len(cols) + 2))
    im = ax.imshow(corr, vmin=-1, vmax=1, cmap="coolwarm")
    fig.colorbar(im, ax=ax, shrink=0.8)
    ax.set_xticks(range(len(cols)), cols, rotation=90)
    ax.set_yticks(range(len(cols)), cols)
    fig.tight_layout()
    return fig


def plot_time_trends(df: pd.DataFrame,
                     time_col: str = "season",
                     group_col: str = "batter_id",
                     velo_col: str = "exit_velo",
                     sample: int = 50):
    """
    Plot mean exit-velo over time for a random sample of batters.
    """
    batters = df[group_col].unique()
    chosen = np.random.choice(batters,
                              min(sample, len(batters)),
                              replace=False)
    fig, ax = plt.subplots(figsize=(8, 4))
    for b in chosen:
        series = (
            df[df[group_col] == b]
            .groupby(time_col)[velo_col]
            .mean()
        )
        ax.plot(series.index, series.values, alpha=0.3)
    ax.set_xlabel(time_col)
    ax.set_ylabel(velo_col)
    ax.set_title("Sample batter exit-velo over time")
    fig.tight_layout()
    return fig


def summarize_numeric_vs_target(
    df: pd.DataFrame,
    numeric_cols: list[str] | None = None,
    target_col: str = "exit_velo",
) -> pd.DataFrame:
    """
    Summarise each numeric predictor against the target.

    Returns a DataFrame indexed by feature with:
      n          – number of non‑null pairs
      pearson_r  – Pearson correlation coefficient
    """
    # --- Pull fresh lists from the schema every time -----------------
    groups = cols.as_dict()

    if numeric_cols is None:
        numeric_cols = groups.get("numerical", [])

    # --- Clean the list ---------------------------------------------
    numeric_cols = [
        c for c in numeric_cols
        if c != target_col and c in df.columns      # ❶ exclude target, ❷ guard
    ]

    records = []
    for col in numeric_cols:
        sub = df[[col, target_col]].dropna()
        if sub.empty:               # skip columns that are all‑NA
            continue
        r = sub[col].corr(sub[target_col])
        records.append({"feature": col, "n": len(sub), "pearson_r": r})

    result = (
        pd.DataFrame.from_records(records)
        .set_index("feature")
        .sort_values("pearson_r", ascending=False)
    )

    print("\n=== Numeric vs target correlations ===")
    print(result)

    return result


def plot_numeric_vs_target(
    df: pd.DataFrame,
    numeric_cols: list[str] | None = None,
    target_col: str = "exit_velo",
):
    """
    Scatter plots of each numeric predictor vs the target with r‑value in title.
    """
    summary = summarize_numeric_vs_target(df, numeric_cols, target_col)
    for feature, row in summary.iterrows():
        plt.figure(figsize=(6, 4))
        plt.scatter(
            df[feature], df[target_col],
            alpha=0.3, s=5, edgecolors="none"
        )
        plt.title(f"{feature} vs {target_col}  (r = {row['pearson_r']:.2f})")
        plt.xlabel(feature)
        plt.ylabel(target_col)
        plt.tight_layout()
        plt.show()



def summarize_categorical_vs_target(
    df: pd.DataFrame,
    cat_cols: list[str] | None = None,
    target_col: str = "exit_velo"
) -> dict[str, pd.DataFrame]:
    """
    For each categorical feature, returns a DataFrame of:
      count, mean, median, std of the target by category.
    """
    groups = get_column_groups()
    if cat_cols is None:
        cat_cols = groups.get("categorical", [])

    summaries: dict[str, pd.DataFrame] = {}
    for col in cat_cols:
        stats = (
            df
            .groupby(col)[target_col]
            .agg(count="count", mean="mean", median="median", std="std")
            .sort_values("count", ascending=False)
        )
        print(f"\n=== {col} vs {target_col} summary ===")
        print(stats)
        summaries[col] = stats
    return summaries


def plot_categorical_vs_target(
    df: pd.DataFrame,
    cat_cols: list[str] | None = None,
    target_col: str = "exit_velo"
):
    """
    For each categorical feature, draw a box‑plot of the target by category.
    """
    groups = get_column_groups()
    if cat_cols is None:
        cat_cols = groups.get("categorical", [])

    for col in cat_cols:
        plt.figure(figsize=(6, 4))
        df.boxplot(column=target_col, by=col, vert=False,
                   grid=False, patch_artist=True)
        plt.title(f"{target_col} by {col}")
        plt.suptitle("")           # remove pandas' automatic suptitle
        plt.xlabel(target_col)
        plt.tight_layout()
        plt.show()



def examine_and_filter_by_sample_size(
    df: pd.DataFrame,
    count_col: str = "exit_velo",
    group_col: str = "batter_id",
    season_col: str = "season",
    percentile: float = 0.05,
    min_count: int | None = None,
    filter_df: bool = False,
) -> tuple[dict[int, pd.DataFrame], pd.DataFrame | None]:
    """
    For each season:
      - compute per-batter count, mean, std of `count_col`
      - pick cutoff: min_count if provided, else the `percentile` quantile
      - print diagnostics
      - plot histograms *safely* (drops NaNs first)
    Returns:
      - summaries: dict season → per-batter summary DataFrame
      - filtered_df: if filter_df, the original df filtered to batters ≥ cutoff
    """
    summaries: dict[int, pd.DataFrame] = {}
    mask_keep: list[pd.Series] = []

    for season, sub in df.groupby(season_col):
        # 1) per-batter summary (count *non-NA* exit_velo)
        summary = (
            sub
            .groupby(group_col)[count_col]
            .agg(count="count", mean="mean", std="std")
            .sort_values("count")
        )
        summaries[season] = summary

        # 2) determine cutoff
        cutoff = min_count if min_count is not None else int(summary["count"].quantile(percentile))
        small = summary[summary["count"] < cutoff]
        large = summary[summary["count"] >= cutoff]

        # 3) diagnostics
        print(f"\n=== Season {season} (cutoff = {cutoff}) ===")
        print(f"  small (<{cutoff} events): {len(small)} batters")
        print(small[["count","mean","std"]].describe(), "\n")
        print(f"  large (≥{cutoff} events): {len(large)} batters")
        print(large[["count","mean","std"]].describe())

        # 4) **safe plotting**: drop NaNs, skip if nothing to plot
        small_means = small["mean"].dropna()
        large_means = large["mean"].dropna()

        if small_means.empty and large_means.empty:
            print(f"  ⚠️  Season {season}: no valid per-batter means to plot")
        else:
            plt.figure(figsize=(8, 3))
            if not small_means.empty:
                plt.hist(small_means, bins=30, alpha=0.6, label=f"n<{cutoff}")
            if not large_means.empty:
                plt.hist(large_means, bins=30, alpha=0.6, label=f"n≥{cutoff}")
            plt.title(f"Season {season}: per-batter EV means")
            plt.xlabel("Mean exit_velo")
            plt.legend()
            plt.tight_layout()
            plt.show()

        # 5) build mask to keep only large-sample batters
        if filter_df:
            keep_ids = large.index
            mask_keep.append(
                (df[season_col] == season) &
                (df[group_col].isin(keep_ids))
            )

    # 6) combine masks and filter
    filtered_df = None
    if filter_df and mask_keep:
        combined = pd.concat(mask_keep, axis=1).any(axis=1)
        filtered_df = df[combined].copy()

    return summaries, filtered_df



def hypothesis_test(df, feature, target="exit_velo", test_type="anova"):
    """
    Perform hypothesis tests for feature significance.
    """
    if test_type == "anova":
        groups = [df[df[feature] == cat][target] for cat in df[feature].unique()]
        F, p = f_oneway(*groups)
        print(f"ANOVA: F={F:.3f}, p={p:.3e}")
        return F, p
    elif test_type == "ttest":
        group1 = df[df[feature] == 0][target]
        group2 = df[df[feature] == 1][target]
        t, p = ttest_ind(group1, group2)
        print(f"T-test: t={t:.3f}, p={p:.3e}")
        return t, p


# ------------------------------------------------------------------
#  NEW: robust outlier flagging
# ------------------------------------------------------------------
def flag_outliers_iqr(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    iqr_mult: float = 1.5,
) -> pd.Series:
    """
    🔹 Why it matters – extreme EVs (>120 mph or <40 mph) can distort
      skew / variance estimates used in hierarchical priors.

    Returns a boolean Series (True = *suspect* outlier) using the
    classic IQR rule: value < Q1 − k·IQR  or  > Q3 + k·IQR.
    """
    q1, q3 = df[velo_col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - iqr_mult * iqr, q3 + iqr_mult * iqr
    mask = (df[velo_col] < lower) | (df[velo_col] > upper)
    n = int(mask.sum())
    print(f"> Outlier flag ({velo_col}): {n} rows outside [{lower:.1f}, {upper:.1f}]")
    return mask



# ------------------------------------------------------------------
#  NEW: EV distribution summary + QQ plot
# ------------------------------------------------------------------
def ev_distribution_summary(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    bins: int = 40,
):
    """
    🔹 Why it matters – confirms right‑skew & heavy‑tail nature of EV
      so you can choose a skew‑normal or Student‑t likelihood.

    Prints skew/kurtosis, shows histogram, KDE, CDF & QQ (if scipy).
    """
    data = df[velo_col].dropna()
    print(
        f"Skewness = {stats.skew(data):.2f},  "
        f"Kurtosis = {stats.kurtosis(data, fisher=False):.2f}"
    )
    fig, ax = plt.subplots(1, 3, figsize=(12, 3))
    ax[0].hist(data, bins=bins, density=True, alpha=0.7)
    data.plot(kind="kde", ax=ax[0], linewidth=2)
    ax[0].set_title("Histogram & KDE")

    # empirical CDF
    ecdf_x = np.sort(data)
    ecdf_y = np.arange(1, len(ecdf_x) + 1) / len(ecdf_x)
    ax[1].plot(ecdf_x, ecdf_y)
    ax[1].set_title("Empirical CDF")

    # QQ vs normal
    from scipy import stats as _st
    _st.probplot(data, dist="norm", plot=ax[2])
    ax[2].set_title("QQ‑plot vs Normal")
    plt.tight_layout()
    return fig


# ------------------------------------------------------------------
#  NEW: Year/era trend diagnostic
# ------------------------------------------------------------------
def year_trend_ev(
    df: pd.DataFrame,
    season_col: str = "season",
    velo_col: str = "exit_velo",
    ci: bool = True,
):
    """
    🔹 Why it matters – detects ball‑era shifts (e.g. 2019 “juiced”,
      2021 “deadened”) so forecasts for 2024 use correct baseline.

    Produces a table & line plot of mean/median EV per season.
    """
    g = df.groupby(season_col)[velo_col]
    stats_df = g.agg(mean="mean", median="median", n="count")
    print("\n=== Exit‑velo by season ===")
    print(stats_df)

    fig, ax = plt.subplots(figsize=(7, 3))
    stats_df["mean"].plot(ax=ax, marker="o", label="Mean EV")
    stats_df["median"].plot(ax=ax, marker="s", label="Median EV")
    if ci:
        sem = g.sem()
        ax.fill_between(
            stats_df.index,
            stats_df["mean"] - 1.96 * sem,
            stats_df["mean"] + 1.96 * sem,
            alpha=0.2,
            label="95% CI (mean)"
        )
    ax.set_ylabel(velo_col)
    ax.set_title("Seasonal trend in exit velocity")
    ax.legend()
    plt.tight_layout()
    return stats_df, fig



# ------------------------------------------------------------------
#  REPLACE old red_flag_level_effect  → clearer name & doc
# ------------------------------------------------------------------
def league_level_effect(
    df: pd.DataFrame,
    level_col: str = "level_abbr",
    velo_col: str = "exit_velo",
) -> tuple[float | None, float | None]:
    """
    🔹 Why it matters – confirms MLB vs Triple‑A (etc.) differences to
      justify hierarchical level effects in the model.

    One‑way ANOVA of `exit_velo` across `level_col`.
    Returns (F, p) or (None, None) if SciPy unavailable.
    """
    if not _HAS_STATS_LIBS:
        print("> SciPy unavailable – falling back to group summary")
        print(df.groupby(level_col)[velo_col].describe())
        return None, None

    groups = [df[df[level_col] == lv][velo_col].dropna()
              for lv in df[level_col].unique()]
    f_val, p_val = stats.f_oneway(*groups)
    print(f"> Level effect ANOVA: F={f_val:.3f}, p={p_val:.3e}")
    return f_val, p_val





# debugs:
def summarize_categorical_missingness(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each categorical column (ordinal + nominal), compute:
      - original_null_count / pct
      - imputed_missing_count / pct
    Safely handles pandas.Categorical by first adding 'MISSING' to its categories.
    """
    cols    = _ColumnSchema()
    cat_cols = cols.ordinal() + cols.nominal()
    summary = []
    n = len(df)

    for col in cat_cols:
        ser = df[col]
        orig_null = ser.isna().sum()

        # If it's a Categorical, add 'MISSING' as a valid category
        if is_categorical_dtype(ser):
            ser = ser.cat.add_categories(['MISSING'])

        # Count rows that would become 'MISSING'
        imputed_missing = ser.fillna('MISSING').eq('MISSING').sum()

        summary.append({
            'column': col,
            'original_null_count':   orig_null,
            'original_null_pct':     orig_null / n,
            'imputed_missing_count': imputed_missing,
            'imputed_missing_pct':   imputed_missing / n,
        })

    return pd.DataFrame(summary)





if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)

    # --- inspect nulls in the raw data ---
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")

    
    df_fe = feature_engineer(df)

    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())

    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:     ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    # use list‐comprehension to drop target(s) from numerical features
    numericals_without_y = [c for c in numericals if c not in cols.target()]

    # Filter out rows with nulls in the target column(s)
    target_cols = cols.target()
    df_filtered = df.dropna(subset=target_cols)

    # Check for nulls in the filtered data
    null_counts_filtered = df_filtered.isnull().sum()
    null_counts_filtered = null_counts_filtered[null_counts_filtered > 0]
    if null_counts_filtered.empty:
        print("✅  No missing values in filtered data.")
    else:
        print("=== Filtered data null counts ===")
        for col, cnt in null_counts_filtered.items():
            print(f" • {col!r}: {cnt} missing")

    print("\n===== check on small samples =====")
    summaries, _ = examine_and_filter_by_sample_size(df_filtered, percentile=0.05)
    summaries, df_filtered = examine_and_filter_by_sample_size(
        df_filtered, percentile=0.05, min_count=15, filter_df=False
    )

    
    # Example usage
    print("\n===== NULLS CHECK =====")
    check_nulls(df_fe)
    
    print("\n===== QUICK PULSE CHECK =====")
    quick_pulse_check(df_fe)
    
    print("\n===== RED FLAGS CHECK =====")
    check_red_flags(df_fe)
    
    print("\n===== AGE EFFECT ANALYSIS =====")
    diag_age_effect(df_fe, age_col="age")
    
    print("\n===== TIME SERIES ANALYSIS =====")
    diag_time_series_dw(df_fe)
    
    print("\n===== PLOTTING =====")
    fig1 = plot_distributions(df_fe, by="hit_type")
    fig2 = plot_correlations(df_fe, numericals)  # Using cols schema
    fig3 = plot_time_trends(df_fe, sample=20)


    # — Numeric features —
    num_summary = summarize_numeric_vs_target(df_fe)
    plot_numeric_vs_target(df_fe)

    # — Categorical features —
    cat_summary = summarize_categorical_vs_target(df_fe)
    plot_categorical_vs_target(df_fe)

    # Example: Test if age has significant effect
    hypothesis_test(df_fe, feature="age_bin", test_type="anova")
    
    
    league_level_effect(df_fe)
    year_trend_ev(df_fe)
    flag_outliers_iqr(df_fe)
    ev_distribution_summary(df_fe)
    # _optional_dw_check(df_fe)   # only if you still care


    summary_df = summarize_categorical_missingness(df_fe)
    print(summary_df.to_markdown(index=False))
    
    # uniques of outcome
    print("outcome uniques:=========================")
    print(df_fe["outcome"].unique())
    # uniques of hit_type
    print("hit_type uniques:=========================")
    print(df_fe["hit_type"].unique())


In [ ]:
%%writefile src/features/data_prep.py
import pandas as pd
import numpy as np

# ─────────────────────────────────────────────────────────────────────────────
def compute_clip_bounds(
    series: pd.Series,
    *,
    method: str = "quantile",
    quantiles: tuple[float,float] = (0.01,0.99),
    std_multiplier: float = 3.0,
) -> tuple[float,float]:
    """
    Compute (lower, upper) but do not apply them.
    """
    s = series.dropna()
    if method == "quantile":
        return tuple(s.quantile(list(quantiles)).to_list())
    if method == "mean_std":
        mu, sigma = s.mean(), s.std()
        return mu - std_multiplier*sigma, mu + std_multiplier*sigma
    if method == "iqr":
        q1, q3 = s.quantile([0.25,0.75])
        iqr = q3 - q1
        return q1 - 1.5*iqr, q3 + 1.5*iqr
    raise ValueError(f"Unknown method {method}")


def clip_extreme_ev(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    lower: float | None = None,
    upper: float | None = None,
    *,
    method: str = "quantile",
    quantiles: tuple[float, float] = (0.01, 0.99),
    std_multiplier: float = 3.0,
    debug: bool = False,
) -> pd.DataFrame:
    """
    Clip exit velocities to [lower, upper].

    If lower/upper are None, compute them from the data using one of:
      - method="quantile": use df[velo_col].quantile(quantiles)
      - method="mean_std":  use mean ± std_multiplier * std
      - method="iqr":       use [Q1 - 1.5*IQR, Q3 + 1.5*IQR]

    When debug=True, prints counts & percentages of values that will be clipped.
    """
    df = df.copy()
    series = df[velo_col].dropna()

    # 1) infer bounds if not given
    if lower is None or upper is None:
        if method == "quantile":
            low_q, high_q = quantiles
            lower_, upper_ = series.quantile([low_q, high_q]).to_list()
        elif method == "mean_std":
            mu, sigma = series.mean(), series.std()
            lower_, upper_ = mu - std_multiplier * sigma, mu + std_multiplier * sigma
        elif method == "iqr":
            q1, q3 = series.quantile([0.25, 0.75])
            iqr = q3 - q1
            lower_, upper_ = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        else:
            raise ValueError(f"Unknown method '{method}' for clip_extreme_ev")
        lower  = lower  if lower  is not None else lower_
        upper  = upper  if upper  is not None else upper_

    # 2) debug: count how many will be clipped
    if debug:
        total   = len(series)
        n_low   = (series < lower).sum()
        n_high  = (series > upper).sum()
        print(f"[clip_extreme_ev] lower={lower:.2f}, upper={upper:.2f}")
        print(f"  → {n_low:,} / {total:,} ({n_low/total:.2%}) below lower")
        print(f"  → {n_high:,} / {total:,} ({n_high/total:.2%}) above upper")

    # 3) actually clip
    df[velo_col] = df[velo_col].clip(lower, upper)
    return df


# ─────────────────────────────────────────────────────────────────────────────
def filter_bunts_and_popups(
    df: pd.DataFrame,
    hit_col: str = "hit_type",
    debug: bool = False
) -> pd.DataFrame:
    """
    Drop bunts and pop-ups from the DataFrame.
    If debug=True, prints how many rows were removed.
    """
    df = df.copy()
    initial = len(df)
    mask = ~df[hit_col].str.upper().isin(["BUNT", "POP_UP"])
    filtered = df[mask]
    if debug:
        removed = initial - len(filtered)
        print(f"[filter_bunts_and_popups] dropped {removed:,} rows "
              f"({removed/initial:.2%}) bunts/pop-ups")
    return filtered




# ─────────────────────────────────────────────────────────────────────────────
# Smoke test / CLI entry
# ─────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.data.ColumnSchema import _ColumnSchema
    from src.features.eda import summarize_categorical_missingness
    from src.features.feature_engineering import feature_engineer
    # from src.features.data_prep import filter_and_clip
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    # use list‐comprehension to drop target(s) from numerical features
    numericals_without_y = [c for c in numericals if c not in cols.target()]
    

    df_fe = feature_engineer(df)

    summary_df = summarize_categorical_missingness(df_fe)
    print(summary_df.to_markdown(index=False))
    
    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    
    # filter out bunts and popups
    df_fe = filter_bunts_and_popups(df_fe)
    # chekc on bunts and popups
    print(df_fe["hit_type"].unique())
    print(df_fe["outcome"].unique())

    debug = True
    TARGET = cols.target()
    lower, upper = compute_clip_bounds(
        df[TARGET],
        method="quantile",            # default: 1st/99th percentile
        quantiles=(0.01, 0.99),
    )
    if debug:
        total = len(df)
        n_low = (df[TARGET] < lower).sum()
        n_high= (df[TARGET] > upper).sum()
        print(f"[fit_preprocessor] clipping train EV to [{lower:.2f}, {upper:.2f}]")
        print(f"  → {n_low:,}/{total:,} ({n_low/total:.2%}) below")
        print(f"  → {n_high:,}/{total:,} ({n_high/total:.2%}) above")
    df_clipped = clip_extreme_ev(df, lower=lower, upper=upper)

    print("Final rows after filter & clip:", len(df_clipped))



# Preprocessing

In [4]:
%%writefile src/features/preprocess.py
"""
Preprocessing module for exit velocity pipeline.
Supports multiple model types (linear, XGBoost, PyMC, etc.) with
automatic ordinal-category detection from the data.
"""
import pandas as pd
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from src.data.ColumnSchema import _ColumnSchema
from sklearn.model_selection import train_test_split
from src.features.data_prep import filter_bunts_and_popups, compute_clip_bounds, clip_extreme_ev


# ───────────────────────────────────────────────────────────────────────
# Numeric & nominal pipelines (unchanged)
# ───────────────────────────────────────────────────────────────────────
numeric_linear = Pipeline([
    ('impute', SimpleImputer(strategy='median', add_indicator=True)),
    ('scale', StandardScaler()),
])
numeric_iterative = Pipeline([
    ('impute', IterativeImputer(random_state=0, add_indicator=True)),
    ('scale', StandardScaler()),
])
nominal_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='MISSING')),
    ('encode', OneHotEncoder(drop='first', handle_unknown='ignore')),
])

# ───────────────────────────────────────────────────────────────────────
# Helper function for domain cleaning to reduce duplication
# ───────────────────────────────────────────────────────────────────────
def filter_and_clip(
    df: pd.DataFrame,
    lower: float = None,
    upper: float = None,
    quantiles: tuple[float, float] = (0.01, 0.99),
    debug: bool = False
) -> pd.DataFrame:
    """
    Clean the dataset by:
    1. Filtering out bunts and popups
    2. Clipping extreme exit velocity values
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame to clean
    lower : float, optional
        Lower bound for clipping, computed from data if None
    upper : float, optional
        Upper bound for clipping, computed from data if None
    quantiles : tuple[float, float], optional
        Quantiles to use if computing bounds from data
    debug : bool, optional
        Whether to print diagnostic information
        
    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame (copy of input)
    tuple[float, float]
        The (lower, upper) bounds used for clipping
    """
    cols = _ColumnSchema()
    TARGET = cols.target()
    
    # 1. Filter out bunts and popups
    df = filter_bunts_and_popups(df, debug=debug)
    
    # 2. Compute bounds if not provided
    if lower is None or upper is None:
        lower_computed, upper_computed = compute_clip_bounds(
            df[TARGET], method="quantile", quantiles=quantiles
        )
        lower = lower if lower is not None else lower_computed
        upper = upper if upper is not None else upper_computed
    
    # 3. Clip extreme values
    df = clip_extreme_ev(df, lower=lower, upper=upper, debug=debug)
    
    return df, (lower, upper)

# ───────────────────────────────────────────────────────────────────────
# Dynamic preprocess functions
# ───────────────────────────────────────────────────────────────────────
# ─────────────────────────────────────────────────────────────────────────────
# 1) fit_preprocessor (with train‐set‐only bound computation + clip)  
# ─────────────────────────────────────────────────────────────────────────────
def fit_preprocessor(
    df: pd.DataFrame,
    model_type: str = "linear",
    debug: bool = False,
    quantiles: tuple[float, float] = (0.01, 0.99),
    max_safe_rows: int = 200000  # Added parameter for warning threshold
) -> tuple[np.ndarray, pd.Series, ColumnTransformer]:
    """
    Fit a preprocessing pipeline to transform raw data into model-ready features.
    
    IMPORTANT: This function should ONLY be called on TRAINING data, not the full dataset.
    The bounds for clipping are computed from this data and applied to future transforms.
    
    Parameters
    ----------
    df : pd.DataFrame
        Training data to fit the preprocessor on. Should NOT include test data.
    model_type : str, default="linear"
        Type of model to prepare for ("linear" or other for iterative imputation)
    debug : bool, default=False
        Whether to print diagnostic information
    quantiles : tuple[float, float], default=(0.01, 0.99)
        Quantiles to use for clipping extreme values
    max_safe_rows : int, default=200000
        Warning threshold for suspiciously large datasets
        
    Returns
    -------
    X_mat : np.ndarray
        Transformed feature matrix
    y : pd.Series
        Target values
    ct : ColumnTransformer
        Fitted transformer
        
    Warns
    -----
    UserWarning
        If the input DataFrame has more rows than max_safe_rows
    """
    # Warn if input is suspiciously large (might be entire dataset)
    if len(df) > max_safe_rows:
        warnings.warn(
            f"Dataset has {len(df)} rows which exceeds threshold of {max_safe_rows}. "
            "You should call fit_preprocessor ONLY on TRAINING data, not the full dataset.",
            UserWarning
        )

    cols = _ColumnSchema()
    TARGET = cols.target()

    # ------------------------------------------------------------------ #
    # 0) domain filter  ➔ 1) clip target
    # ------------------------------------------------------------------ #
    df, (lower, upper) = filter_and_clip(df, quantiles=quantiles, debug=debug)

    # ------------------------------------------------------------------ #
    # 2) split feature sets + coerce numerics
    # ------------------------------------------------------------------ #
    num_feats = [c for c in cols.numerical() if c != TARGET]
    ord_feats = cols.ordinal()
    nom_feats = cols.nominal()

    df[num_feats] = df[num_feats].apply(pd.to_numeric, errors="coerce")
    X = df[num_feats + ord_feats + nom_feats].copy()
    y = df[TARGET]

    # ------------------------------------------------------------------ #
    # 3) **NEW** – make ordinals plain strings before mutation
    # ------------------------------------------------------------------ #
    X[ord_feats] = X[ord_feats].astype("string")

    # safe masking
    X.loc[:, ord_feats] = X.loc[:, ord_feats].mask(X.loc[:, ord_feats].isna(), np.nan)

    ordinal_categories = [[*X[c].dropna().unique(), "MISSING"] for c in ord_feats]
    ordinal_pipe = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value="MISSING")),
        ("encode", OrdinalEncoder(
            categories=ordinal_categories,
            handle_unknown="use_encoded_value",
            unknown_value=-1,
            dtype="int32",
        )),
    ])

    # ------------------------------------------------------------------ #
    # 4) numeric pipeline selection
    # ------------------------------------------------------------------ #
    imp_cls, imp_kwargs = (
        (SimpleImputer, {"strategy": "median", "add_indicator": True})
        if model_type == "linear"
        else (IterativeImputer, {"random_state": 0, "add_indicator": True})
    )

    numeric_pipe = Pipeline([
        ("impute", imp_cls(**imp_kwargs)),
        ("scale",  StandardScaler()),
    ])

    # ------------------------------------------------------------------ #
    # 5) ColumnTransformer
    # ------------------------------------------------------------------ #
    ct = ColumnTransformer(
        [("num", numeric_pipe, num_feats),
         ("ord", ordinal_pipe, ord_feats),
         ("nom", nominal_pipe, nom_feats)],
        remainder="drop",
        verbose_feature_names_out=False,
    )
    ct.lower_, ct.upper_ = lower, upper
    X_mat = ct.fit_transform(X, y)
    return X_mat, y, ct




# ─────────────────────────────────────────────────────────────────────────────
def transform_preprocessor(
    df: pd.DataFrame,
    transformer: ColumnTransformer,
) -> tuple[np.ndarray, pd.Series]:
    """
    Transform new data using a fitted preprocessor.
    
    Parameters
    ----------
    df : pd.DataFrame
        New data to transform
    transformer : ColumnTransformer
        Fitted transformer from fit_preprocessor
        
    Returns
    -------
    X_mat : np.ndarray
        Transformed feature matrix
    y : pd.Series
        Target values
    """
    cols, TARGET = _ColumnSchema(), _ColumnSchema().target()

    # Domain filter & clip using the bounds from the fitted transformer
    df, _ = filter_and_clip(df, lower=transformer.lower_, upper=transformer.upper_)

    # rebuild lists & coerce numerics
    num_feats = [c for c in cols.numerical() if c != TARGET]
    ord_feats = cols.ordinal()
    nom_feats = cols.nominal()
    df[num_feats] = df[num_feats].apply(pd.to_numeric, errors="coerce")

    X = df[num_feats + ord_feats + nom_feats].copy()

    # **NEW** – ensure ordinals are strings, then fill unseen→"MISSING"
    X[ord_feats] = X[ord_feats].astype("string")
    X.loc[:, ord_feats] = X.loc[:, ord_feats].where(X.loc[:, ord_feats].notna(), "MISSING")

    y = df[TARGET]
    X_mat = transformer.transform(X)
    return X_mat, y


def inverse_transform_preprocessor(
    X_trans: np.ndarray,
    transformer: ColumnTransformer
) -> pd.DataFrame:
    """
    Invert each block of a ColumnTransformer back to its original features,
    based on the exact column lists we passed in.
    """
    import numpy as np
    import pandas as pd
    import warnings

    # 1) Flatten the lists we gave each transformer to recover original feature order
    orig_features: list[str] = []
    for name, _, cols in transformer.transformers_:
        if cols == 'drop':
            continue
        orig_features.extend(cols)

    parts = []
    start = 0
    n_rows = X_trans.shape[0]

    # 2) For each transformer, slice & inverse-transform
    for name, trans, cols in transformer.transformers_:
        if cols == 'drop':
            continue

        fitted = transformer.named_transformers_[name]

        # how many columns did it produce?  (we only use this for slicing)
        dummy = np.zeros((1, len(cols)))
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            try:
                out = fitted.transform(dummy)
            except Exception:
                out = dummy
        n_out = out.shape[1]

        # now slice the real block
        block = X_trans[:, start : start + n_out]
        start += n_out

        # apply inverse_transform to get back original columns
        if trans == 'passthrough':
            inv = block
        elif name == 'num':
            scaler = fitted.named_steps['scale']
            inv_full = scaler.inverse_transform(block)
            inv = inv_full[:, :len(cols)]
        else:
            # ordinal or nominal pipelines
            if isinstance(fitted, Pipeline):
                last = list(fitted.named_steps.values())[-1]
                inv = last.inverse_transform(block)
            else:
                inv = fitted.inverse_transform(block)

        parts.append(pd.DataFrame(inv, columns=cols, index=range(n_rows)))

    # 3) Concatenate & reorder
    df_orig = pd.concat(parts, axis=1)
    return df_orig[orig_features]



def prepare_for_mixed_and_hierarchical(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the rows *and* adds convenience covariates expected by the
    hierarchical and mixed-effects models.
    
    Parameters
    ----------
    df : pd.DataFrame
        Feature-engineered DataFrame
        
    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame with domain filtering and category encoding
    """
    cols = _ColumnSchema()
    TARGET = cols.target()

    # Apply domain cleaning with default quantiles
    df, _ = filter_and_clip(df.copy())

    # Category coding for later random-effects
    df["batter_id"] = df["batter_id"].astype("category")
    return df











# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    df_fe = feature_engineer(df)
    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()


    train_df, test_df = train_test_split(df_fe, test_size=0.2, random_state=42)
    # run with debug prints
    X_train, y_train, tf = fit_preprocessor(train_df, model_type='linear', debug=True)
    X_test,  y_test      = transform_preprocessor(test_df, tf)

    print("Processed shapes:", X_train.shape, X_test.shape)

    # Example of inverse transform: 
    print("==========Example of inverse transform:==========")
    df_back = inverse_transform_preprocessor(X_train, tf)
    print("\n✅ Inverse‐transformed head (should mirror your original X_train):")
    print(df_back.head())
    print("Shape:", df_back.shape, "→ original X_train shape before transform:", X_train.shape)




Overwriting src/features/preprocess.py


# feature selection

In [ ]:
%%writefile src/features/feature_selection.py
import pandas as pd

# ── NEW: model and importance imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap

from pathlib import Path
from src.data.load_data import load_and_clean_data
from src.features.feature_engineering import feature_engineer
from src.data.ColumnSchema import _ColumnSchema
# ── NEW: shapash and shapiq imports
from shapash import SmartExplainer
import shapiq
from sklearn.utils import resample

def train_baseline_model(X, y):
    """
    Fit a RandomForestRegressor on X, y.
    Returns the fitted model.
    """
    # You can adjust hyperparameters as needed
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X, y)
    return model



def compute_permutation_importance(
    model,
    X: pd.DataFrame,
    y: pd.Series,
    n_repeats: int = 10,
    n_jobs: int = 1,
    max_samples: float | int = None,
    random_state: int = 42,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Compute permutation importances with controlled resource usage.
    
    Parameters
    ----------
    model : estimator
        Fitted model implementing .predict and .score.
    X : pd.DataFrame
        Features.
    y : pd.Series or array
        Target.
    n_repeats : int
        Number of shuffles per feature.
    n_jobs : int
        Number of parallel jobs (avoid -1 on Windows).
    max_samples : float or int, optional
        If float in (0,1], fraction of rows to sample.
        If int, absolute number of rows to sample.
    random_state : int
        Seed for reproducibility.
    verbose : bool
        Print debug info if True.
        
    Returns
    -------
    pd.DataFrame
        Columns: feature, importance_mean, importance_std.
        Sorted descending by importance_mean.
    """
    # Debug info
    if verbose:
        print(f"⏳ Computing permutation importances on {X.shape[0]} rows × {X.shape[1]} features")
        print(f"   n_repeats={n_repeats}, n_jobs={n_jobs}, max_samples={max_samples}")

    # Subsample if requested
    X_sel, y_sel = X, y
    if max_samples is not None:
        if isinstance(max_samples, float):
            nsamp = int(len(X) * max_samples)
        else:
            nsamp = int(max_samples)
        if verbose:
            print(f"   Subsampling to {nsamp} rows for speed")
        X_sel, y_sel = resample(X, y, replace=False, n_samples=nsamp, random_state=random_state)

    try:
        result = permutation_importance(
            model,
            X_sel, y_sel,
            n_repeats=n_repeats,
            random_state=random_state,
            n_jobs=n_jobs,
        )
    except OSError as e:
        # Graceful fallback to single job
        if verbose:
            print(f"⚠️  OSError ({e}). Retrying with n_jobs=1")
        result = permutation_importance(
            model,
            X_sel, y_sel,
            n_repeats=n_repeats,
            random_state=random_state,
            n_jobs=1,
        )

    # Build and sort DataFrame
    importance_df = (
        pd.DataFrame({
            "feature": X.columns,
            "importance_mean": result.importances_mean,
            "importance_std": result.importances_std,
        })
        .sort_values("importance_mean", ascending=False)
        .reset_index(drop=True)
    )
    if verbose:
        print("✅ Permutation importances computed.")
    return importance_df


def compute_shap_importance(model, X, nsamples=100):
    """
    Compute mean absolute SHAP values per feature.
    Returns a DataFrame sorted by importance.
    """
    # DeepExplainer or TreeExplainer for tree-based models
    explainer = shap.TreeExplainer(model)
    # sample for speed
    X_sample = X.sample(n=min(nsamples, len(X)), random_state=42)
    shap_values = explainer.shap_values(X_sample)
    # For regression, shap_values is a 2D array
    mean_abs_shap = pd.DataFrame({
        "feature": X_sample.columns,
        "shap_importance": np.abs(shap_values).mean(axis=0),
    })
    mean_abs_shap = mean_abs_shap.sort_values("shap_importance", ascending=False).reset_index(drop=True)
    return mean_abs_shap



def filter_permutation_features(
    importance_df: pd.DataFrame,
    threshold: float
) -> list[str]:
    """
    Return features whose permutation importance_mean exceeds threshold.
    """
    kept = importance_df.loc[
        importance_df["importance_mean"] > threshold, "feature"
    ]
    return kept.tolist()


def filter_shap_features(
    importance_df: pd.DataFrame,
    threshold: float
) -> list[str]:
    """
    Return features whose shap_importance exceeds threshold.
    """
    kept = importance_df.loc[
        importance_df["shap_importance"] > threshold, "feature"
    ]
    return kept.tolist()


def select_final_features(
    perm_feats: list[str],
    shap_feats: list[str],
    mode: str = "intersection"
) -> list[str]:
    """
    Combine permutation and SHAP feature lists.
    mode="intersection" for features in both lists,
    mode="union" for features in either list.
    """
    set_perm = set(perm_feats)
    set_shap = set(shap_feats)
    if mode == "union":
        final = set_perm | set_shap
    else:
        final = set_perm & set_shap
    # return sorted for reproducibility
    return sorted(final)



def load_final_features(
    file_path: str = "data/models/features/final_features.txt"
) -> list[str]:
    """
    Read the newline-delimited feature names file and return as a list.
    """
    with open(file_path, "r") as fp:
        return [line.strip() for line in fp if line.strip()]


def filter_to_final_features(
    df: pd.DataFrame,
    file_path: str = "data/models/features/final_features.txt"
) -> pd.DataFrame:
    """
    Given a feature-engineered DataFrame, load the final feature list,
    then return df[ ID_cols + final_features + [target] ].
    """
    # load the feature names
    final_feats = load_final_features(file_path)
    cols = _ColumnSchema()

    keep = cols.id() + final_feats + [cols.target()]
    missing = set(keep) - set(df.columns)
    if missing:
        raise ValueError(f"Cannot filter: missing columns {missing}")
    return df[keep].copy()





if __name__ == "__main__":
    # --- existing loading & schema logic ---
    raw_path = Path("data/Research Data Project/Research Data Project/exit_velo_project_data.csv")
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")
    df_fe = feature_engineer(df)
    print("Raw →", df.shape, "//  Feature-engineered →", df_fe.shape)
    print(df_fe.head())

    cols = _ColumnSchema()
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:     ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    numericals_without_y = [c for c in numericals if c not in cols.target()]

    # ── STEP 1: fully preprocess the engineered DataFrame ──
    from src.features.preprocess import fit_preprocessor, inverse_transform_preprocessor

    # fit_preprocessor returns (X_matrix, y, fitted_transformer)
    X_np, y, preproc = fit_preprocessor(df_fe, model_type='linear', debug=False)

    # Use the same index that y carries (only non-bunt, non-NA rows)
    idx = y.index
    
    # turn that into a DataFrame with the same column names:
    feat_names = preproc.get_feature_names_out()
    X = pd.DataFrame(X_np, columns=feat_names, index=idx)
    print(f"✅ Preprocessed feature matrix: {X.shape[0]} rows × {X.shape[1]} cols")

    # (optional) confirm inverse transform lines up:
    df_back = inverse_transform_preprocessor(X_np, preproc)
    df_back.index = idx
    print("✅ inverse_transform round-trip (head):")
    print(df_back.head())

    # ── STEP 2: train & compute importances on *that* X ──
    print("\nTraining baseline model…")
    model = train_baseline_model(X, y)

    print("\n🔍 Permutation Importances:")
    perm_imp = compute_permutation_importance(
        model, X, y,
        n_repeats=10,
        n_jobs=2,            # test small parallelism
        max_samples=0.5,     # test subsampling
        verbose=True
    )
    print(perm_imp)


    print("\n🔍 SHAP Importances:")
    shap_imp = compute_shap_importance(model, X)
    print(shap_imp)

    # ── STEP 3: threshold & select your final features ──
    perm_thresh = 0.01
    shap_thresh = 0.01
    perm_feats = filter_permutation_features(perm_imp, perm_thresh)
    shap_feats = filter_shap_features(shap_imp, shap_thresh)
    final_feats = select_final_features(perm_feats, shap_feats, mode="intersection")
    print(f"\nFinal preprocessed feature list ({len(final_feats)}):")
    print(final_feats)

    # ── STEP 4: build & save a dataset with just those features + target + IDs ──
    df_final = pd.concat([
        df_fe[cols.id()],
        df_fe[[cols.target()]],
        X[final_feats]
    ], axis=1)
    print("Final dataset shape:", df_final.shape)

    Path("data/models/features/final_features.txt").write_text("\n".join(final_feats))
    print("✔️ Saved feature list to final_features.txt")


    # Demo: filter the full df_fe back to just those features
    df_filtered = filter_to_final_features(df_fe)
    print("Filtered to final features shape:", df_filtered.shape)


# model choices

see modelling_choices.txt

In [ ]:
%%writefile src/models/linear.py

"""
Fast linear baselines (OLS and Ridge).

Usage
-----
>>> from src.models.linear import fit_ridge
>>> fitted, rmse = fit_ridge(train_df, val_df)
"""
from __future__ import annotations
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline


def _split_xy(df: pd.DataFrame):
    X = df.drop(columns=["exit_velo"])
    y = df["exit_velo"]
    return X, y


def fit_ridge(X_tr: pd.DataFrame,
              y_tr: pd.DataFrame,
              X_te: pd.DataFrame,
              y_te: pd.DataFrame,
              alpha: float = 1.0):
    """
    Returns (sklearn Pipeline, RMSE on test set).
    """

    model = Pipeline(
        [("reg" , Ridge(alpha=alpha, random_state=0))]
    ).fit(X_tr, y_tr)

    pred = model.predict(X_te)
    rmse = np.sqrt(np.mean((pred - y_te) ** 2))
    return model, rmse



if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.data.ColumnSchema import _ColumnSchema
    from sklearn.model_selection import train_test_split
    from src.features.preprocess import summarize_categorical_missingness
    from src.features.preprocess import fit_preprocessor, transform_preprocessor, inverse_transform_preprocessor
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)

    # --- inspect nulls in the raw data ---
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")
    df_fe = feature_engineer(df)

    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())

    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    # use list‐comprehension to drop target(s) from numerical features
    numericals_without_y = [c for c in numericals if c not in cols.target()]

    summary_df = summarize_categorical_missingness(df_fe)
    print(summary_df.to_markdown(index=False))


    # check nulls
    print("🛠️  Nulls in X before fit_transform:")
    null_counts = df_fe.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values after feature engineering.")
    else:
        print("=== Null counts post-engineering ===")
        print(null_counts)

    train_df, test_df = train_test_split(df_fe, test_size=0.2, random_state=42)
    
    # only on training data for linear/XGB
    train_df = clip_extreme_ev(train_df)
    #valid_df = clip_extreme_ev(valid_df)
    
    # run with debug prints
    X_train, y_train, tf = fit_preprocessor(train_df, model_type='linear', debug=True)
    X_test,  y_test      = transform_preprocessor(test_df, tf)

        
    print("Processed shapes:", X_train.shape, X_test.shape)

    # Example of inverse transform: 
    print("==========Example of inverse transform:==========")
    df_back = inverse_transform_preprocessor(X_train, tf)
    print("\n✅ Inverse‐transformed head (should mirror your original X_train):")
    print(df_back.head())
    print("Shape:", df_back.shape, "→ original X_train shape before transform:", X_train.shape)
    

    # === NEW: Train & evaluate Ridge regression ===
    model_ridge, rmse_ridge = fit_ridge(X_train, y_train, X_test,  y_test)
    print(f"Ridge regression RMSE: {rmse_ridge:.4f}")


In [ ]:
# %%writefile src/models/gbm.py

"""
Gradient‑boosting baseline (XGBoost regressor).
"""
from __future__ import annotations
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
import optuna
from src.data.ColumnSchema import _ColumnSchema
from xgboost.core import XGBoostError

# ————— Detect GPU support using modern API —————
try:
    XGBRegressor(tree_method="hist", device="cuda")
    GPU_SUPPORT = True
except XGBoostError:
    GPU_SUPPORT = False

def _split_xy(df: pd.DataFrame):
    X = df.drop(columns=["exit_velo"])
    y = df["exit_velo"]
    return X, y

def tune_gbm(X, y, n_trials: int = 50):
    """
    Run an Optuna study to minimize CV RMSE of an XGBRegressor.
    Uses device=cuda if available, else CPU.
    """
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "random_state": 0,
            "n_jobs": -1,
        }
        params.update({"tree_method": "hist"})
        model = XGBRegressor(**params)
        scores = cross_val_score(
            model, X, y,
            scoring="neg_root_mean_squared_error",
            cv=3, n_jobs=-1
        )
        return -scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_trial.params

def fit_gbm(X_tr, y_tr, X_te, y_te, **gbm_kw):
    """
    Train XGBRegressor with optional hyperparams, early stopping,
    and automatic GPU/CPU selection.
    Returns (model, RMSE).
    """
    gbm_default = dict(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.7,
        colsample_bytree=0.6,
        random_state=0,
        n_jobs=-1,
        early_stopping_rounds=50,
    )
    gbm_default.update({"tree_method": "hist"})
    gbm_default.update(gbm_kw)

    model = XGBRegressor(**gbm_default)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_te, y_te)],
        early_stopping_rounds=gbm_default["early_stopping_rounds"],
        verbose=False
    )
    preds = model.predict(X_te)
    rmse = mean_squared_error(y_te, preds, squared=False)
    return model, rmse



# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from features.archive.preprocess import (fit_preprocessor
                                        ,transform_preprocessor
                                        ,inverse_transform_preprocessor)
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    df_fe = feature_engineer(df)
    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()


    train_df, test_df = train_test_split(df_fe, test_size=0.2, random_state=42)
    # run with debug prints
    # Record original feature count before preprocessing
    X_orig = train_df.drop(columns=["exit_velo"])
    orig_shape = X_orig.shape

    # Run preprocessing
    X_train, y_train, tf = fit_preprocessor(train_df, model_type='linear', debug=True)
    X_test,  y_test      = transform_preprocessor(test_df, tf)

    print("Processed shapes:", X_train.shape, X_test.shape)

    # Example of inverse transform: 
    print("==========Example of inverse transform:==========")
    df_back = inverse_transform_preprocessor(X_train, tf)
    print("\n✅ Inverse‐transformed head (should mirror your original X_train):")
    print(df_back.head())
    print(f"Shape: {df_back.shape} → original X_train shape before transform: {orig_shape}")


        

    # === Hyperparameter tuning ===
    best_params = tune_gbm(X_train, y_train, n_trials=50)
    print("Tuned params:", best_params)

    # === Train & evaluate ===
    gbm_model, rmse = fit_gbm(
        X_train, y_train, X_test, y_test, **best_params
    )
    print(f"Tuned XGBoost RMSE: {rmse:.4f}")


/opt/conda/envs/marlins-ds-gpu/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


Dropping rows with NaN in exit_velo
season               0
level_abbr           0
batter_id            0
pitcher_id           0
exit_velo        97979
launch_angle      9130
spray_angle       6227
hangtime         75968
hit_type           201
batter_hand          0
pitcher_hand         0
batter_height        0
pitch_group          0
outcome              0
age                  0
dtype: int64
after rows dropped with NaN in exit_velo
season               0
level_abbr           0
batter_id            0
pitcher_id           0
exit_velo            0
launch_angle         0
spray_angle          0
hangtime         12429
hit_type             0
batter_hand          0
pitcher_hand         0
batter_height        0
pitch_group          0
outcome              0
age                  0
dtype: int64
⚠️  Nulls after target‑filter:
  • hangtime         12,429 (1.00%)
   season level_abbr  batter_id  pitcher_id  exit_velo  launch_angle  \
0    2023        mlb        235        1335    95.7352       47.2362

/workspaces/Marlins_Data_Science_Project/src/features/feature_engineering.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["player_ev_mean50"].fillna(gm, inplace=True)
/workspaces/Marlins_Data_Science_Project/src/features/feature_engineering.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

Raw → (1246545, 15) //  Feature‑engineered → (1246545, 29)
   season level_abbr  batter_id  pitcher_id  exit_velo  launch_angle  \
0    2023        MLB        235        1335    95.7352       47.2362   
1    2023        MLB       3182        1335    95.9380        4.7291   
2    2023        MLB       3856        1988    89.1404      -16.2251   
3    2023        MLB       2017        1988    88.7278       -6.8385   
4    2023        MLB       1594        1988    89.2888        0.5079   

   spray_angle  hangtime     hit_type batter_hand  ...           spray_bin  \
0      -6.4422    6.4960     FLY_BALL           R  ...     (-12.378, 9.48]   
1      -4.8052    0.7806  GROUND_BALL           L  ...     (-12.378, 9.48]   
2      15.2382    0.0311  GROUND_BALL           S  ...     (9.48, 179.962]   
3     -11.5988    0.1215  GROUND_BALL           R  ...     (-12.378, 9.48]   
4     -22.1899    0.3802  GROUND_BALL           R  ...  (-179.87, -12.378]   

   same_hand hand_match pitch_hand_matc

/workspaces/Marlins_Data_Science_Project/src/features/preprocess.py:138: UserWarning: Dataset has 997236 rows which exceeds threshold of 200000. You should call fit_preprocessor ONLY on TRAINING data, not the full dataset.
  warnings.warn(


[filter_bunts_and_popups] dropped 70,284 rows (7.05%) bunts/pop-ups
[clip_extreme_ev] lower=49.01, upper=110.25
  → 9,270 / 926,952 (1.00%) below lower
  → 9,268 / 926,952 (1.00%) above upper
Processed shapes: (926952, 46) (231778, 46)
==========Example of inverse transform:==========


[I 2025-05-06 15:46:06,435] A new study created in memory with name: no-name-009d1576-23a4-486d-8bce-d01de1ebd97f



✅ Inverse‐transformed head (should mirror your original X_train):
   launch_angle  spray_angle  hangtime  height_diff  age_sq  age_centered  \
0       20.8142       1.3858    3.8519      0.81666  686.44          -0.2   
1      -75.5873      24.1692    0.0250      1.81666  817.96           2.2   
2       -4.0382       5.8295    0.1551      3.81666  576.00          -2.4   
3      -24.0564      42.8805    0.0543     -0.18334  823.69           2.3   
4       33.4007      11.4086    5.6830     -3.18334  681.21          -0.3   

   season_centered  level_idx  player_ev_mean50  player_ev_std50  ...  \
0             -2.0        0.0         87.179676        13.061763  ...   
1              2.0        0.0         88.659742        11.356806  ...   
2             -2.0        0.0         89.364194        15.835891  ...   
3              1.0        2.0         90.890562        16.252956  ...   
4              0.0        2.0         88.196544        15.333166  ...   

          age_bin             l

/opt/conda/envs/marlins-ds-gpu/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/opt/conda/envs/marlins-ds-gpu/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant

In [ ]:
%%writefile src/models/mixed.py

"""
Frequentist mixed‑effects model using statsmodels MixedLM.

Formula implemented:
    exit_velo ~ 1 + level_ord + age_centered
              + (1 | batter_id)

We rely on columns already produced by preprocess():
    • level_idx  (0,1,2)   – ordinal
    • age_centered
"""
from __future__ import annotations
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf


def fit_mixed(train: pd.DataFrame,
              test: pd.DataFrame):
    """Return (fitted model, RMSE on test)."""
    # statsmodels wants a *single* DataFrame with all cols
    # so we concatenates and keep row positions for slicing
    combined = pd.concat([train, test], axis=0)
    # ensure categorical dtype
    combined["level_ord"] = combined["level_idx"].astype(int)

    mdl = smf.mixedlm(
        formula="exit_velo ~ 1 + level_ord + age_centered",
        data=combined.iloc[: len(train)],
        groups=combined.iloc[: len(train)]["batter_id"]
    ).fit(reml=False)

    # predict on test set
    pred = mdl.predict(exog=combined.iloc[len(train):])
    true = test["exit_velo"].values
    rmse = np.sqrt(np.mean((pred - true) ** 2))
    return mdl, rmse

if __name__ == "__main__":
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import prepare_for_mixed_and_hierarchical
    from sklearn.model_selection import train_test_split

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    df_fe = feature_engineer(df)

    # Prepare and split
    df_model = prepare_for_mixed_and_hierarchical(df_fe)
    train_df, test_df = train_test_split(df_model, test_size=0.2, random_state=42)
 
    # Fit mixed-effects
    mixed_model, rmse_mixed = fit_mixed(train_df, test_df)
    print(f"Mixed-effects model RMSE: {rmse_mixed:.4f}")

    # In the smoke test section: P-Value Checks for Mixed-Effects Models
    print("Mixed-effects model summary:\n", mixed_model.summary())

In [ ]:
%%writefile src/utils/bayesian_explainability.py
import arviz as az
import shap, numpy as np
import matplotlib.pyplot as plt

# ---------------- Posterior summaries -----------------
def plot_parameter_forest(idata, var_names=None, hdi_prob=0.95):
    """Caterpillar/forest plot of posterior estimates."""
    return az.plot_forest(
        idata,
        var_names=var_names,
        combined=True,
        hdi_prob=hdi_prob,
        kind="forest",
        figsize=(6, len(var_names or idata.posterior.data_vars) * 0.4),
    )

def posterior_table(idata, round_to=2):
    """
    Return a nicely rounded HDI/mean table with significance.
    """
    summary = az.summary(idata, hdi_prob=0.95).round(round_to)
    summary["significant"] = (summary["hdi_2.5%"] > 0) | (summary["hdi_97.5%"] < 0)
    return summary

# ---------------- Posterior‑predictive checks ---------
def plot_ppc(idata, kind="overlay"):
    """Visual PPC (over‑laid densities by default)."""
    return az.plot_ppc(idata, kind=kind, alpha=0.1)

# ---------------- SHAP-based feature importances ------
def shap_explain(predict_fn, background_df, sample_df):
    """
    Model‑agnostic Kernel SHAP on the *posterior mean predictor*.

    predict_fn(df) must return a 1‑D numpy array of predictions.
    """
    explainer = shap.KernelExplainer(predict_fn, background_df)
    shap_values = explainer.shap_values(sample_df, nsamples=200)
    shap.summary_plot(shap_values, sample_df, show=False)
    plt.tight_layout()
    return shap_values


In [1]:
%%writefile src/models/hierarchical_utils.py

import arviz as az
import joblib
from pathlib import Path

def save_model(idata, file_path: str, overwrite: bool = True):
    """Save ArviZ InferenceData to NetCDF."""
    idata.to_netcdf(file_path, engine="h5netcdf", overwrite_existing=overwrite)
    print(f"✔︎ saved model → {file_path}")

def load_model(file_path: str):
    """Load ArviZ InferenceData from NetCDF."""
    idata = az.from_netcdf(file_path, engine="h5netcdf")
    print(f"✔︎ loaded model ← {file_path}")
    return idata

def save_preprocessor(transformer, file_path: str):
    """Save the scikit-learn transformer to a joblib file."""
    joblib.dump(transformer, file_path)
    print(f"✔︎ saved preprocessor → {file_path}")

def load_preprocessor(file_path: str):
    """Load the scikit-learn transformer from a joblib file."""
    transformer = joblib.load(file_path)
    print(f"✔︎ loaded preprocessor ← {file_path}")
    return transformer

if __name__ == "__main__":
    # === Editable settings ===
    # Path to the saved model (NetCDF format)
    MODEL_PATH = "data/models/saved_models/model.nc"
    # Path to the preprocessor
    PREPROC_PATH = "data/models/saved_models/preprocessor.joblib"
    # Input data for prediction (raw CSV with exit velocity data)
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    # Output predictions file (CSV) or set to None to print to console
    OUTPUT_PREDS_2024 = "data/predictions/predictions_2024.csv"  # <-- EDITABLE: set output CSV path or None
    
    model = load_model(MODEL_PATH)
    save_model(model, MODEL_PATH)



Overwriting src/models/hierarchical_utils.py


In [3]:
%%writefile src/utils/posterior.py
# src/utils/posterior.py
import numpy as np
import pandas as pd
import arviz as az

# ── REPLACEMENT: paste over the whole function ─────────────────────────
import json, pathlib, numpy as np, pandas as pd, arviz as az

JSON_GLOBAL = pathlib.Path("data/models/saved_models/global_effects.json")

def posterior_to_frame(idata: az.InferenceData) -> pd.DataFrame:
    """
    Returns a batter‑level summary **AND** writes global effects to JSON.

    File written  ➜  data/models/saved_models/global_effects.json
    """
    post = idata.posterior

    # ------- per‑batter u summaries -----------------------------------
    u   = post["u"]                                         # (chain,draw,batter)
    stats = {
        "u_mean"  : u.mean(("chain","draw")).values,
        "u_sd"    : u.std (("chain","draw")).values,
        "u_q2.5"  : np.percentile(u.values,  2.5, axis=(0,1)),
        "u_q50"   : np.percentile(u.values, 50.0, axis=(0,1)),
        "u_q97.5" : np.percentile(u.values,97.5, axis=(0,1)),
    }
    df = pd.DataFrame({"batter_idx": np.arange(u.shape[-1]), **stats})

    # ------- global effects ------------------------------------------
    mu_mean = post["mu"].mean().item()

    # β_age  ➜ last entry of beta vector (age_centered was added last)
    beta  = post["beta"]
    feat_dim = [d for d in beta.dims if d not in ("chain","draw")][0]
    beta_age = beta.isel({feat_dim: -1}).mean().item()

    beta_level = post["beta_level"].mean(("chain","draw")).values.tolist()
    sigma_b    = post["sigma_b"].mean().item()
    sigma_e    = post["sigma_e"].mean().item()

    global_eff = dict(
        mu_mean=mu_mean,
        beta_age=beta_age,
        beta_level=beta_level,
        sigma_b=sigma_b,
        sigma_e=sigma_e,
        median_age=idata.attrs.get("median_age", 26.0),
        beta=post["beta"].mean(("chain","draw")).values.tolist(),  # Save all beta coefficients
        feature_names=idata.attrs.get("feature_names", [])  # Also save feature names if available
    )

    # ➜  write side‑car JSON (overwrite every run)
    JSON_GLOBAL.write_text(json.dumps(global_eff, indent=2))
    print(f"✔︎ wrote global effects → {JSON_GLOBAL}")

    return df





def align_batter_codes(df_roster: pd.DataFrame,
                       train_cats: pd.Index) -> pd.Series:
    """
    Convert integer batter_ids in *roster* into the categorical codes
    **identical** to what the model saw during training.

    Any unseen batter gets code = -1 (handled later).
    """
    cat = pd.Categorical(df_roster["batter_id"], categories=train_cats)
    return pd.Series(cat.codes, index=df_roster.index)



Overwriting src/utils/posterior.py


In [5]:
%%writefile src/models/hierarchical.py

import pymc as pm
import arviz as az
import numpy as np
from src.data.ColumnSchema import _ColumnSchema
from src.features.preprocess import transform_preprocessor
# ── Attempt to import JAX ──────────────────────────────────────
USE_JAX = True
try:
    import jax
    # Debug: confirm what module is loaded
    print(f"🔍 JAX module: {jax!r}")
    print(f"🔍 JAX path:   {getattr(jax, '__file__', 'builtin')}")
    # Ensure version attribute exists (guards circular-import)
    if not hasattr(jax, "__version__"):
        raise ImportError("jax.__version__ missing—possible circular import")
    print(f"✅ JAX version: {jax.__version__}")
    # Enable 64-bit floats on GPU/CPU
    jax.config.update("jax_enable_x64", True)
except Exception as e:
    USE_JAX = False
    print(f"⚠️  Warning: could not import JAX ({e}). Falling back to CPU sampling.")

import pymc as pm
import arviz as az
import numpy as np
import pandas as pd


# Configure JAX for GPU use and X64 precision
jax.config.update("jax_enable_x64", True)
print("JAX version:", jax.__version__)
print("JAX devices:", jax.devices())
print("GPU count:", jax.device_count("gpu"))
print("Default backend:", jax.default_backend())

import logging, pymc.sampling.jax as pmjax


# ── NEW: fit_bayesian_hierarchical with timing & ETAs ────────────────
import time
from contextlib import contextmanager
from tqdm.auto import tqdm          # auto‑selects rich bar in Jupyter / CLI

@contextmanager
def _timed_section(label: str):
    """Context manager that prints elapsed time for a code block."""
    t0 = time.time()
    yield
    dt = time.time() - t0
    print(f"[{label}] finished in {dt:,.1f} s")




# ------------------------------------------------------------------
def fit_bayesian_hierarchical(
    df_raw,
    transformer,
    batter_idx: np.ndarray,
    level_idx: np.ndarray,
    *,
    feature_list: list[str] | None = None,
    mu_mean: float  = 88.0,
    mu_sd:   float  = 30.0,
    sigma_prior: float = 10.0,
    draws: int      = 200,
    tune:  int      = 200,
    target_accept: float = 0.9,
    verbose: bool   = True,
    sampler: str    = "jax",
):
    cols = _ColumnSchema()
    if feature_list is None:
        feature_list = cols.model_features()

    # 1) design matrix
    X_all, y_ser = transform_preprocessor(df_raw, transformer)
    names = transformer.get_feature_names_out()
    X     = X_all[:, np.isin(names, feature_list)]
    y     = y_ser.values

    # ── CAST TO PYTHON INTS ───────────────────────────────────────────────
    n_bat = int(batter_idx.max()) + 1
    n_lvl = int(level_idx.max()) + 1
    n_feat= int(X.shape[1])

    if verbose:
        print(f"🔍 n_bat={n_bat} ({type(n_bat)}), "
              f"n_lvl={n_lvl} ({type(n_lvl)}), "
              f"n_feat={n_feat} ({type(n_feat)})")

    with pm.Model() as model:
        # ── priors ─────────────────────────────────────────
        mu         = pm.Normal("mu", mu_mean, mu_sd)
        beta_level = pm.Normal("beta_level", 0, 5, shape=n_lvl)
        beta       = pm.Normal("beta",       0, 5, shape=n_feat)
        sigma_b    = pm.HalfNormal("sigma_b", sigma_prior)
        u_raw      = pm.Normal("u_raw", 0, 1, shape=n_bat)
        u          = pm.Deterministic("u", u_raw * sigma_b)

        # ── likelihood ────────────────────────────────────
        theta    = mu + beta_level[level_idx] + pm.math.dot(X, beta) + u[batter_idx]
        sigma_e  = pm.HalfNormal("sigma_e", sigma_prior)
        pm.Normal("y_obs", theta, sigma_e, observed=y)

        # ── sampling ──────────────────────────────────────
        idata = pm.sample(
            draws=draws,
            tune=tune,
            chains=4,
            target_accept=target_accept,
            nuts_sampler="numpyro" if (sampler=="jax" and USE_JAX) else "nuts",
            progressbar=True,
        )
        idata.extend(pm.sample_posterior_predictive(idata, var_names=["y_obs"]))

    # Store feature names on idata for downstream use
    used_feats = [names[i] for i in range(len(names)) if np.isin(names[i], feature_list)]
    idata.attrs["feature_names"] = used_feats

    if verbose:
        print("⚡ First 5 posterior-pred EV samples:",
              idata.posterior_predictive["y_obs"].stack(s=("chain","draw")).values[:5])

    return idata
# ------------------------------------------------------------------





# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # %%writefile src/train_hierarchical.py
    from pathlib import Path
    import pandas as pd, numpy as np, arviz as az
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import (fit_preprocessor,
                                        prepare_for_mixed_and_hierarchical)
    # from src.models.hierarchical import fit_bayesian_hierarchical
    from src.models.hierarchical_utils import save_model, save_preprocessor
    from src.utils.posterior import posterior_to_frame

    RAW   = Path("data/Research Data Project/Research Data Project/exit_velo_project_data.csv")
    OUT_NC = Path("data/models/saved_models/exitvelo_hmc.nc")
    OUT_POST = Path("data/models/saved_models/posterior_summary.parquet")
    OUT_PREPROC = Path("data/models/saved_models/preprocessor.joblib")

    # 1 · prep
    df = load_and_clean_data(RAW)
    df_fe = feature_engineer(df)
    df_model = prepare_for_mixed_and_hierarchical(df_fe)

    _, _, tf = fit_preprocessor(df_model, model_type="linear", debug=False)

    b_idx = df_model["batter_id"].cat.codes.values
    l_idx = df_model["level_idx"].values
    draws_and_tune = 100
    target_accept=0.9
    # 2 · fit
    idata = fit_bayesian_hierarchical(df_model, tf, b_idx, l_idx,
                                    sampler="jax", draws=draws_and_tune, tune=draws_and_tune, target_accept=target_accept)

    idata.attrs["median_age"] = df_model["age"].median()   # ← NEW

    # 3 · persist
    save_model(idata, OUT_NC)
    save_preprocessor(tf, OUT_PREPROC)
    posterior_to_frame(idata).to_parquet(OUT_POST)
    print("✅ training complete – artefacts written:")
    print("   •", OUT_NC)
    print("   •", OUT_POST)
    print("   •", OUT_PREPROC)




Overwriting src/models/hierarchical.py


In [6]:
%%writefile src/models/hierarchical_predict.py

import json
from pathlib import Path

import arviz as az
import pandas as pd
import numpy as np

from src.features.feature_engineering import feature_engineer
from src.features.preprocess import prepare_for_mixed_and_hierarchical
from src.utils.posterior import align_batter_codes


def predict_from_idata(
    df_raw: pd.DataFrame,
    idata: az.InferenceData
) -> pd.DataFrame:
    """
    Generate population-level predictions from a fitted hierarchical model.

    Parameters
    ----------
    df_raw : pd.DataFrame
        Raw roster DataFrame containing at least ['season','batter_id','age','level'].
    idata : arviz.InferenceData
        Fitted model inference data.

    Returns
    -------
    pd.DataFrame
        Input df_raw with added columns:
        - level_idx, age_centered
        - pred_mean (μ + β_level + β_age·age_centered)
    """
    # 1) feature engineering & preprocessing
    df_feat = feature_engineer(df_raw)
    df_model = prepare_for_mixed_and_hierarchical(df_feat)

    # 2) extract posterior-means of the fixed effects
    pm = idata.posterior.mean(dim=("chain", "draw"))
    mu = pm["mu"].values
    beta_level = pm["beta_level"].values
    beta_age = pm["beta_age"].values

    # 3) compute point predictions
    df_model["pred_mean"] = (
        mu[df_model.index]  # mu can be vector per row if modeled that way
        + beta_level[df_model["level_idx"].values]
        + beta_age * df_model["age_centered"].values
    )

    return df_model


def predict_from_summaries(
    roster_csv: Path,
    posterior_parquet: Path,
    global_effects_json: Path,
    output_csv: Path,
    preprocessor_path: Path = Path("data/models/saved_models/preprocessor.joblib")
) -> pd.DataFrame:
    """
    Load your saved summaries + raw roster, merge in random effects,
    compute full point & interval predictions, and write to CSV.

    Returns the full merged DataFrame (with contrib_* and pred_lo95/hi95).
    """
    # 1) load data and preprocessor
    from src.models.hierarchical_utils import load_preprocessor
    
    df_post = pd.read_parquet(posterior_parquet)   # tidy posterior summary
    df_roster = pd.read_csv(roster_csv)            # season, batter_id, age
    
    try:
        transformer = load_preprocessor(preprocessor_path)
        have_transformer = True
    except Exception as e:
        print(f"⚠️ Could not load preprocessor: {e}")
        print("Continuing without feature effects (predictions may be biased)")
        have_transformer = False

    # 2) global (static) effects
    glob = json.loads(global_effects_json.read_text())
    post_mu    = glob["mu_mean"]
    beta_age   = glob["beta_age"]
    beta_level = glob["beta_level"][2]      # MLB index
    med_age    = glob["median_age"]
    
    # Get feature coefficients if available
    beta_features = glob.get("beta", [])
    feature_names = glob.get("feature_names", [])

    # 3) merge random effects
    df_roster["batter_idx"] = align_batter_codes(df_roster, df_post["batter_idx"])
    df = df_roster.merge(df_post, on="batter_idx", how="left")
    global_sigma_b = df_post["u_sd"].mean()
    df["u_mean"] = df["u_mean"].fillna(0.0)
    df["u_sd"]   = df["u_sd"].fillna(global_sigma_b)

    # 4) Feature engineering and preprocessing (if transformer is available)
    feature_effects = 0.0
    if have_transformer and beta_features and feature_names:
        from src.features.feature_engineering import feature_engineer
        from src.features.preprocess import transform_preprocessor, prepare_for_mixed_and_hierarchical
        
        try:
            # Apply same preprocessing pipeline as during training
            df_feat = feature_engineer(df_roster)
            df_model = prepare_for_mixed_and_hierarchical(df_feat)
            
            # Transform to get design matrix
            X_all, _ = transform_preprocessor(df_model, transformer)
            
            # Get feature values from transformer
            all_names = transformer.get_feature_names_out()
            feature_indices = [i for i, name in enumerate(all_names) if name in feature_names]
            
            if feature_indices:
                X = X_all[:, feature_indices]
                feature_effects = np.dot(X, beta_features)
                print(f"✅ Applied effects from {len(feature_indices)} features")
            else:
                print("⚠️ Could not match feature names")
        except Exception as e:
            print(f"⚠️ Error applying feature preprocessing: {e}")
            print("Continuing with base effects only")

    # 5) compute predictions + contributions + 95% intervals
    df["age_centered"] = df["age"] - med_age
    df["pred_mean"] = (
        post_mu
        + beta_level
        + beta_age * df["age_centered"]
        + df["u_mean"]
        + feature_effects  # Add the feature effects
    )
    df["contrib_age"]   = beta_age * df["age_centered"]
    df["contrib_level"] = beta_level
    df["contrib_u"]     = df["u_mean"]
    df["contrib_features"] = feature_effects
    z95 = 1.96
    df["pred_lo95"] = df["pred_mean"] - z95 * df["u_sd"]
    df["pred_hi95"] = df["pred_mean"] + z95 * df["u_sd"]

    # 6) export
    df[[
        "season", "batter_id", "pred_mean", "pred_lo95", "pred_hi95", 
        "contrib_age", "contrib_level", "contrib_u", "contrib_features"
    ]].to_csv(output_csv, index=False)
    print(f"📄 Predictions written → {output_csv}")

    return df


if __name__ == "__main__":
    # Example CLI usage for your 2024 predictions
    BASE          = Path("data/models/saved_models")
    P_SUMMARY     = BASE / "posterior_summary.parquet"
    P_GLOBAL      = BASE / "global_effects.json"
    P_PREPROC     = BASE / "preprocessor.joblib"
    POSTERIOR = Path("data/models/saved_models/posterior_summary.parquet")
    ROSTER_INPUT    = Path("data/Research Data Project/Research Data Project/exit_velo_validate_data.csv")
    OUTPUT_CSV    = Path("data/predictions/exitvelo_predictions_2024.csv")


    predict_df = predict_from_summaries(
        roster_csv=ROSTER_INPUT,
        posterior_parquet=P_SUMMARY,
        global_effects_json=P_GLOBAL,
        output_csv=OUTPUT_CSV,
        preprocessor_path=P_PREPROC
    )


    print(predict_df.head()[[
        "batter_id", "pred_mean", 
        "contrib_level", "contrib_age", "contrib_u", "contrib_features"
    ]])


Overwriting src/models/hierarchical_predict.py


In [ ]:
%%writefile src/utils/validation.py
"""
Generic K‑fold validator.

• Works for sklearn Pipelines *or* PyMC idata.
• Decides how to extract predictions based on
  the object returned by `fit_func`.
"""

import numpy as np
from sklearn.model_selection import KFold
import arviz as az

from __future__ import annotations
import pandas as pd
from sklearn.model_selection import KFold
from typing import Callable, List, Union
import statsmodels as sm

def _split_xy(df: pd.DataFrame):
    X = df.drop(columns=["exit_velo"])
    y = df["exit_velo"]
    return X, y


def _rmse(a, b):
    return np.sqrt(np.mean((a - b) ** 2))


def run_kfold_cv(
    fit_func: Callable[[pd.DataFrame, pd.DataFrame], tuple],
    df: pd.DataFrame,
    k: int = 5,
    random_state: int = 0,
    **fit_kw
) -> List[float]:
    """
    fit_func(train_df, test_df, **fit_kw) -> (model_or_idata, rmse)
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    rmses: List[float] = []

    for train_idx, test_idx in kf.split(df):
        train, test = df.iloc[train_idx], df.iloc[test_idx]
        _, rmse = fit_func(train, test, **fit_kw)
        rmses.append(rmse)

    return rmses


# helper to score a *single* train/test split for idata
def rmse_pymc(idata: az.InferenceData, test_df: pd.DataFrame) -> float:
    """Posterior mean vs truth."""
    pred = (
        idata.posterior_predictive["y_obs"]
        .mean(("chain", "draw"))
        .values
    )
    return _rmse(pred, test_df["exit_velo"].values)

def run_kfold_cv(fit_func, df, k=5, random_state=0, **fit_kwargs):
    """
    Apply `fit_func(train_df, **fit_kwargs)` then evaluate on held-out.
    Returns list of held-out log_likelihoods or RMSEs.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    scores = []
    for train_idx, test_idx in kf.split(df):
        train, test = df.iloc[train_idx], df.iloc[test_idx]
        idata = fit_func(train, **fit_kwargs)

        # posterior predictive on test
        ppc = az.from_pymc(posterior_predictive=idata, model=None)
        pred_mean = ppc.posterior_predictive["y_obs"].mean(("chain","draw")).values
        true = test["exit_velo"].values
        rmse = np.sqrt(((pred_mean - true)**2).mean())
        scores.append(rmse)
    return scores

def posterior_predictive_check(idata, df, batter_idx):
    """
    Plot observed vs. simulated exit-velo histograms.
    """
    import matplotlib.pyplot as plt
    obs = df["exit_velo"].values
    sim = idata.posterior_predictive["y_obs"].stack(samples=("chain","draw")).values.flatten()

    fig, ax = plt.subplots(1,2,figsize=(8,3))
    ax[0].hist(obs, bins=30); ax[0].set_title("Observed")
    ax[1].hist(sim, bins=30); ax[1].set_title("Simulated")
    fig.tight_layout()
    return fig




def prediction_interval(model, X, alpha=0.05, method='linear'):
    """
    Compute prediction intervals for a model.
    """
    if method == 'linear':
        # For OLS and Ridge
        X_const = sm.add_constant(X)
        preds = model.get_prediction(X_const)
        pred_int = preds.conf_int(alpha=alpha)
        return preds.predicted_mean, pred_int[:, 0], pred_int[:, 1]
    elif method == 'bayesian':
        # For Bayesian models
        hdi = az.hdi(model, hdi=1 - alpha)
        return (
            hdi.posterior_predictive.y_obs.sel(hdi=f"{alpha/2*100}%"),
            hdi.posterior_predictive.y_obs.sel(hdi=f"{(1-alpha/2)*100}%")
        )
    elif method == 'gbm':
        # For XGBoost quantile regression
        lower = model.predict(X, pred_contribs=False, iteration_range=(0, model.best_iteration))
        upper = model.predict(X, pred_contribs=False, iteration_range=(0, model.best_iteration))
        return lower, upper  # Replace with actual quantile regression
    else:
        raise ValueError("Method not supported")

# Example for bootstrapping GBM
def bootstrap_prediction_interval(model, X, n_bootstraps=1000, alpha=0.05):
    preds = np.zeros((n_bootstraps, X.shape[0]))
    for i in range(n_bootstraps):
        indices = np.random.choice(X.shape[0], X.shape[0], replace=True)
        preds[i] = model.predict(X[indices])
    lower = np.percentile(preds, 100 * alpha / 2, axis=0)
    upper = np.percentile(preds, 100 * (1 - alpha / 2), axis=0)
    return lower, upper


# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import prepare_for_mixed_and_hierarchical

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    predict_path = "data/Research Data Project/Research Data Project/exit_velo_validate_data.csv.csv"
    df = load_and_clean_data(raw_path)
    df_fe = feature_engineer(df)

    # Prepare the DataFrame
    df_model = prepare_for_mixed_and_hierarchical(df_fe)

    # Extract arrays for PyMC
    batter_idx   = df_model["batter_id"].cat.codes.values
    level_idx    = df_model["level_idx"].values
    age_centered = df_model["age_centered"].values

    # Fit the Bayesian hierarchical model
    idata = fit_bayesian_hierarchical(
        df_model, batter_idx, level_idx, age_centered,
        mu_prior=90, sigma_prior=5,
        sampler="jax",   #  <-- GPU NUTS
        draws=1000, tune=1000
    )

    print(idata)
    
    posterior_predictive_check(idata, df_model, df_model.batter_id.cat.codes.values)
    
    # For Bayesian model:
    lower, upper = prediction_interval(idata, test_df, method='bayesian')
    print(f"Bayesian 95% Prediction Interval: {lower.mean():.2f}–{upper.mean():.2f} mph")

    # For Ridge model:
    pred, lower, upper = prediction_interval(model_ridge, X_test, method='linear')
    print(f"Ridge 95% Prediction Interval: {lower[0]:.2f}–{upper[0]:.2f} mph")

In [ ]:
%%writefile src/train.py
"""
Train / compare four families on a 70‑30 split.

Run:
    python -m src.train
"""
from __future__ import annotations
import pandas as pd
from sklearn.model_selection import train_test_split

from src.data.load_data import load_and_clean_data
from src.features.preprocess import preprocess

from src.models.linear import fit_ridge
from src.models.gbm   import fit_gbm
from src.models.mixed import fit_mixed
from src.models.hierarchical import fit_bayesian_hierarchical

RAW_PATH = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"


def main():
    df_raw   = load_and_clean_data(RAW_PATH)
    df_clean = preprocess(df_raw)

    train_df, test_df = train_test_split(
        df_clean, test_size=0.30, random_state=42, stratify=df_clean["level_abbr"]
    )

    # ––– A  Ridge  –––––––––––––––––––––––––––––––
    _, rmse_ridge = fit_ridge(train_df, test_df)
    print(f"Ridge RMSE ……  {rmse_ridge:5.2f} mph")

    # ––– B  Gradient‑Boost  ––––––––––––––––––––––
    _, rmse_gbm = fit_gbm(train_df, test_df)
    print(f"XGBoost RMSE … {rmse_gbm:5.2f} mph")

    # ––– C  Mixed‑Effects  –––––––––––––––––––––––
    _, rmse_mixed = fit_mixed(train_df, test_df)
    print(f"Mixed‑LM RMSE  {rmse_mixed:5.2f} mph")

    # ––– D  Bayesian Hierarchical (quick sample) –
    idata = fit_bayesian_hierarchical(
        train_df,
        batter_idx=train_df.batter_id.astype("category").cat.codes.values,
        level_idx=train_df.level_idx.values,
        age_centered=train_df.age_centered.values,
        mu_prior=90,
        sigma_prior=5,
        draws=500, tune=500   # short run for demo
    )
    from src.utils.validation import rmse_pymc
    rmse_bayes = rmse_pymc(idata, test_df)
    print(f"PyMC RMSE ……  {rmse_bayes:5.2f} mph")


if __name__ == "__main__":
    main()



In [ ]:
# %%writefile src/models/model_shap_reports.py
# ── NEW: shapash and shapiq imports
from shapash import SmartExplainer
import shapiq

def generate_shapash_report(
    model,
    X,
    y,
    features_dict: dict | None = None,
    preprocessing: object | None = None,
    report_path: str = "shapash_report.html"
):
    """
    Instantiate Shapash SmartExplainer, compile with data, and
    generate both a live app and a standalone HTML report.

    Parameters:
    - model: trained ML model (supports .predict)
    - X: pd.DataFrame, input features
    - y: pd.Series or array, true target values
    - features_dict: optional mapping {col: label} for display
    - preprocessing: optional transformer with inverse_transform
    - report_path: file path to save HTML report
    """
    # 1️⃣ Create the explainer
    xpl = SmartExplainer(
        model=model,
        features_dict=features_dict or {c: c for c in X.columns},
        preprocessing=preprocessing
    )
    # 2️⃣ Compile dataset for Shapash
    y_pred = model.predict(X)
    xpl.compile(
        x=X,
        y_pred=y_pred,
        y_target=y,
        additional_data=None
    )
    # 3️⃣ Launch interactive app (optional; returns a Flask app)
    # app = xpl.run_app()
    # 4️⃣ Generate standalone HTML report
    xpl.generate_report(
        output_file=report_path,
        title_story="Model Explainability Report",
        title_description="Auto-generated by Shapash",
        x_train=None, y_train=None, y_test=X, metrics=[]
    )
    # Return the explainer for further interaction
    return xpl

def compute_shapiq_interactions(
    model,
    X,
    sample_size: int = 100,
    max_order: int = 2
):
    """
    Use shapiq to compute Shapley Interaction values up to `max_order`
    for up to `sample_size` observations.

    Parameters:
    - model: trained ML model
    - X: pd.DataFrame of features
    - sample_size: how many rows to explain
    - max_order: maximum interaction order (e.g., 2 for pairwise)
    
    Returns:
    - interaction_values: shapiq InteractionValues object
    """
    # 1️⃣ Sample data for performance
    X_sample = X.sample(n=min(len(X), sample_size), random_state=42).to_numpy()
    # 2️⃣ Instantiate the explainer
    explainer = shapiq.TabularExplainer(
        model=model,
        data=X_sample,
        index="k-SII",   # or "SV" for standard Shapley values
        max_order=max_order
    )
    # 3️⃣ Explain the first sample
    interaction_values = explainer.explain(X_sample[0], budget=256)
    return interaction_values



if __name__ == "__main__":